In [1]:
import pandas as pd 
import numpy as np
import re
import os
import shutil
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print('import succed!')

import succed!


In [2]:
input_path = r'/Users/edz/Documents/yandi_data/AI_data'
input_path_file = r'/patient3.csv'

In [3]:
df_amd= pd.read_csv(input_path + input_path_file,sep=',',index_col=None,header=None)
print(df_amd.shape)   # 现有库存数据中“阅片诊断”不为空的数据容量

(250625, 16)


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 修改列名
df_amd.columns = ['来自检查id','送阅时间','病例医院名称','阅片医院名称','病历医生id','病例医生名称','阅片医生id','阅片医生名称',
                '病史','全身病','病例诊断','影像描述','阅片诊断','质控影像描述','质控阅片诊断','主诉、病例检查、处理、处方json串']
df_amd.tail(2) #校验列名修改成功

,来自检查id,送阅时间,病例医院名称,阅片医院名称,病历医生id,病例医生名称,阅片医生id,阅片医生名称,病史,全身病,病例诊断,影像描述,阅片诊断,质控影像描述,质控阅片诊断,主诉、病例检查、处理、处方json串
250623,11018815,2019-03-04 14:54:11,河北省保定市东风社区卫生服务中心,至真阅片中心,4752,白月红,82,白海霞,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；视网膜血管走行正常；黄斑区未见明显异常。,双眼眼底未见明显异常,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."
250624,11018943,2019-03-07 09:09:27,昆明市西山区永昌社区卫生服务中心,至真阅片中心,65,李玲玲,12,崔馨,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；左眼及右眼可见部分视网膜血管走行正常，未见异常新生血...,右眼眼底未见明显异常；左眼眼底窥不清,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."


In [5]:
df_amd.duplicated(subset='来自检查id').sum()  #判断指定Series的行数据是否存在重复项
# 或者 df_qgy['来自检查id'].duplicated().sum() 

0

In [6]:
# 自定义”全角转半角“函数
def strQ2B(ustring):
    """把字符串全角转半角"""
    ss = []
    for s in ustring:   # ustring=['后发障','AION','无法诊断'],s=list中依次取得的各个字符串
        rstring = ""
#         print(s) 
        for uchar in s:  # s='后发障',uchar='后发障'中依次取得的各个字符
            inside_code = ord(uchar)
#             print(inside_code)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
#             print(rstring)
        ss.append(rstring)
#         print(ss)
    return ss
#     return ''.join(ss)

In [7]:
#调用函数，实现全角转半角
df_amd['阅片诊断'] = strQ2B(df_amd['阅片诊断'])  # ''.join用于合并字符，df_amd.at['阅片诊断'] = ''.join(strQ2B(str_i))
df_amd.loc[:5,'阅片诊断']

0        右眼大视杯,视网膜动脉硬化
1          双眼眼底未见明显异常。
2           双眼视网膜动脉硬化。
3    双眼DR 中度\n双眼drusen
4                   待诊
5           眼底影像未见明显异常
Name: 阅片诊断, dtype: object

In [8]:
# 插入'阅_gai'列
df_amd.insert(df_amd.columns.get_loc('阅片诊断')+1,'阅_gai',df_amd['阅片诊断'])

## 去除空格、括号、波浪线等
\s表示空格，b = re.compile('\s')表示匹配空格；df_tw.at[i,'阅_gai'] = re.compile('\s').sub('',str_i)
匹配到空格之后替换为“”（即：去除空格）

In [9]:
# 去除字符串中的“\n|\t|\r|~|——”
def kong_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('\n|\t|\r|~|——|(|)|,|、|。|\?').findall(str_i)) > 0: # 去除字符串中的“\n|\t|\r|~|——”
#             print(str_i)
            str_i_1 = str_i.replace('\n','').replace('\r','').replace('\t','').replace(',','').replace('。','').replace('、','')    # 去除\n|\t|\r和,以及。   
            str_i_2 = str_i_1.replace('~','-').replace('——','-')  # 改~和————为短中线
            str_i_3 = re.compile('\?').sub('',str_i_2)
            str_i_4 = re.compile(';').sub('',str_i_3)
            df.at[i,'阅_gai'] = str_i_4.replace('(','').replace(')','')  # 去除括号
#             print('__{}'.format(df.at[i,'阅_gai']))
        else:
            pass

    check_df = df[df['阅_gai'].str.contains('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在无效标点".format(check_df.shape[0]))
        kong_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [10]:
kong_fuc(df_amd)    # 调用去除无效标点的函数

转化结束！！！


In [11]:
# 去除空格
def kong1_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('\s').findall(str_i)) > 0:
    #         print(str_i)
            df.at[i,'阅_gai'] = re.compile('\s').sub('',str_i)  # 去除字符串首尾的空格
    #         print('__{}'.format(df.at[i,'阅_gai']))
        else:
            pass

    check_df = df[df['阅_gai'].str.contains('\s')]  #校验
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在空格".format(check_df.shape[0]))
        kong1_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [12]:
kong1_fuc(df_amd)

转化结束！！！


In [13]:
########### 校验是否还存在空格
for i in range(df_amd.shape[0]):
    str_i = str(df_amd.at[i,'阅_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;').findall(str_i)) > 0:
        print(str_i)
        df_amd.at[i,'阅_gai'] = re.compile('\?').sub('',str_i)  # 去除字符串首尾的空格
        print('__{}'.format(df_amd.at[i,'阅_gai']))
    else:
        pass

In [14]:
# 增加“分隔符”的自定义函数
def add_function(x):
    import re
    list_x=[i.start() for i in re.finditer('双|左|右', x)]
#     print(list_x)

    for i in range(len(list_x)):
        x = list(x)
    #     print(list_x[i])
        x.insert(list_x[i]+i,'!')
        x ="".join(x)
#     print(x)
    return x

In [15]:
#调用函数，增加分隔符
for m in range(df_amd.shape[0]):
    df_amd.at[m,'阅_gai'] = add_function(df_amd.at[m,'阅_gai'])

print(df_amd.loc[:5,'阅_gai'])  #校验

0       !右眼大视杯视网膜动脉硬化
1         !双眼眼底未见明显异常
2          !双眼视网膜动脉硬化
3    !双眼DR中度!双眼drusen
4                  待诊
5          眼底影像未见明显异常
Name: 阅_gai, dtype: object


In [16]:
## 缺少左右眼的——添加“双”标签
# 处理字符串没有“左/右/双”的情形
def lr_shuang_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split('!',str_i)
            str_i_list = list(filter(None,str_tempt))
            str_new_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('(右|左|双)').findall(str_i_list[j])) == 0:
#                     print(str_i_list)
                    str_new_list.append('双' + str_i_list[j])

                else:
                    str_new_list.append(str_i_list[j])
            df.at[i,'阅_gai'] = '!'.join(str_new_list[m] for m in range(len(str_new_list)))
#             print("___{}".format(df.at[i,'阅_gai']))

In [17]:
lr_shuang_fuc(df_amd)  ## 调用添加“双”标签的函数

In [18]:
############## 验证“双”补全
for i in range(df_amd.shape[0]):
    str_i = str(df_amd.at[i,'阅_gai'])
    if str_i == 'nan':
        pass
    else:
        str_tempt = re.split('!',str_i)
        str_i_list = list(filter(None,str_tempt))
        str_new_list = []
        for j in range(len(str_i_list)):
            if len(re.compile('(双|左|右)').findall(str_i_list[j])) == 0:
                print(str_i_list[j])

In [19]:
print("阅_gai——的规格：{}".format(df_amd['阅_gai'].shape[0]))
print("阅片诊断——的规格：{}".format(df_amd['阅片诊断'].shape[0]))

阅_gai——的规格：250625
阅片诊断——的规格：250625


# 通用字符串的名称归一化

In [20]:
# 定义归一化函数——单个标签替换
def single_replace_fuc(patten_x, repl_x,string_x):
    strinfo_0 = re.compile(patten_x, re.I)
    if len(strinfo_0.findall(string_x)) > 0:
#         print(string_x)
        string_x = re.sub(re.compile(patten_x, flags=re.I), repl_x, string_x)
#         print("__{}".format(string_x))
    else:
        pass
    
    return string_x

In [21]:
####定义”双眼\|右眼\|左眼“转为”双|左|右“的转化函数
#双、左、右眼——名称统一
# 为解决调用一次函数，会出现的“修改不完全的问题”，可以多次调用函数处理遗留的未匹配数据。
def lrs_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        else:
            str_i_1 = single_replace_fuc('双眼', '双', str_i)  #str_i_1/2/3均为中间变量,调用函数 single_replace_fuc
            str_i_2 = single_replace_fuc('左眼', '左', str_i_1)
            str_i_3 = single_replace_fuc('右眼', '右', str_i_2)
            df.at[i,'阅_gai'] =  str_i_3

    check_df = df[df['阅_gai'].str.contains('双眼|右眼|左眼')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据包含双眼|右眼|左眼".format(check_df.shape[0]))
        lrs_normal_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [22]:
lrs_normal_fuc(df_amd)    # 调用“”双眼\|右眼\|左眼“转为”双|左|右“的转化函数

剩余23289条数据包含双眼|右眼|左眼
剩余14条数据包含双眼|右眼|左眼
转化结束！！！


In [23]:
####定义Drusen拼写错误的转化函数
#Drusen拼写错误——名称统一
for i in range(df_amd.shape[0]):
    str_i = str(df_amd.at[i,'阅_gai'])
    if str_i == 'nan':
        pass
    else:
        a  = 'Drusen'
        strinfo_0 = re.compile('Drusne', re.I)
        strinfo_1 = re.compile('Druse', re.I)   # re.I 表示忽略大小写
        strinfo_2 = re.compile('Drusen', re.I)
        if len(strinfo_0.findall(str_i,re.I))>0 :  # 
#             print(str_i)
            df_amd.at[i,'阅_gai'] = strinfo_0.sub(a,str_i)
#             print(df_amd.at[i,'阅_gai'])
        elif len(strinfo_1.findall(str_i))>0 and len(strinfo_2.findall(str_i))==0 :  # 排除“Drusen”
#             print(str_i)
            df_amd.at[i,'阅_gai'] = strinfo_1.sub(a,str_i)
#             print(df_amd.at[i,'阅_gai'])
        else:
            pass

## 黄斑变性AMD的名称归一化

### 黄斑病变和黄斑变性的区别？
医生常说的黄斑病变和黄斑变性不是一回事儿。黄斑病变是眼科常见可致盲的疾病之一，它不是一个独立的疾病，而是导致视网膜黄斑区发生病理改变的一组疾病的统称。黄斑变性是黄斑病变的一种，黄斑病变包括黄斑区出血、黄斑裂孔、黄斑前膜、黄斑变性等，黄斑变性只是黄斑病变中的一个小概念。

黄斑病变是很多种黄斑部疾病的统称，比较常见的黄斑病变包括黄斑变性、黄斑裂孔、黄斑前膜、玻璃体黄斑牵拉综合征等等这些黄斑部的疾病，都被称为黄斑病变。其中黄斑变性是黄斑病变中一个比较多见的疾病，一般黄斑变性分为两大类，一类是干性黄斑变性，一类是湿性黄斑变性。
#### 黄斑病变的分类：
常见的有老年性黄斑变性、高度近视导致的黄斑病变、遗传因素导致的黄斑病变等。由此可见，黄斑变性只是黄斑病变的一种。


(STGD)又称青少年型遗传性黄斑变性
 陈旧黄斑病变 ：陈旧性黄斑病变是以前有过黄斑病变治愈后的表现，陈旧性病变是指某种疾病恢复后留下的痕迹
 
  黄斑病变高度近视、高度近视黄斑病变、黄斑变性高度近视性、黄斑病变外伤性、外伤性黄斑病变
  
#### 针对性治疗黄斑病变效果更好

治疗黄斑变性，希玛眼科医院采用光动力治疗（PDT）及玻璃体腔内注射抗血管内皮生长因子药物治疗，可以有效稳定病情，血管内皮生长因子药物对老年黄斑退化具有显著的疗效。

治疗黄斑前膜，现时并没有任何可有效治疗黄斑前膜，手术可谓是唯一出路。黄斑前膜手术可透过微创方式进行玻璃体切除手术，伤口毋须缝线；除了剥除黄斑前膜外，有不少眼科医生更加入剥除内置膜的一步，有研究显示剥除内置膜可减低黄斑前膜复发的机会，手术效果也可能更佳，而且愈早接受手术，完全清除黄斑水肿及使黄斑点回复原貌的机会也愈高。

治疗黄斑裂孔，玻璃体切除手术仍是主要的治疗方法。通常第一期的黄斑裂孔尚不需要手术治疗；当裂孔恶化至第二至四期就需要考虑手术，其中第二期病变中，视神经组织损失尚不明显，如及早手术封闭裂孔，视力恢复程度较佳。而如果病变恶化至第三及第四期，病人视力多已受到较大破坏，即使手术后裂孔封闭，视力恢复可能亦会未如理想。

现代黄斑裂孔手术已经进入微创无缝线时代，除了切除玻璃体，撕掉有疤痕的视网膜内界膜也是主流方法。透过撕掉内界膜，可以更彻底化解拉扯的压力。另外，还会在眼内灌注特别的气体（多数是C3F8）以顶压裂孔，使其与后方视网膜组织保持贴合，而令裂孔能更好地愈合。

#### 黄斑变性是黄斑病变的一种，黄斑病变包括黄斑区出血、黄斑裂孔、黄斑前膜、黄斑变性等，黄斑变性只是黄斑病变中的一个小概念。

In [24]:
df_amd[df_amd['阅_gai'].str.contains('继发(.*)黄斑前膜',re.I)]['阅_gai'].value_counts()  
# 黄斑病变AMD 干性AMD 湿性AMD  黄斑病变湿性AMD  双AMD!右湿性 双黄斑陈旧病变干性AMD  !双AMD干性白内障 左AMD黄斑地图样萎缩 
# 双AMD!右瘢痕期  黄斑病变AMD瘢痕 黄斑病变AMDPCV   左湿性AMD部分瘢痕形成 AMD瘢痕期可能性大  右黄斑病变中浆湿性AMD  !双干性AMD中期
# 黄斑病变中浆  陈旧性黄斑病变  双高度近视眼底改变!双黄斑病变  黄斑病变CNV 双黄斑病变玻璃疣  !左黄斑病变中心性浆液性脉络膜视网膜病变
# !右黄斑病变中渗 中渗：中心性渗出性视网膜病变。而黄斑病变主要是黄斑区病变  双高度近视黄斑病变  双黄斑病变干性AMD  左黄斑病变CNV
# !左黄斑病变外伤性  !左黄斑病变裂孔   双黄斑病变陈旧中浆   双黄斑病变stargardt病    外伤黄斑病变  黄斑病变Best  外伤性黄斑病变
# 右黄斑病变{AMD} 左遗传性黄斑变性  老年黄斑变性  老年性黄斑变性干性  左不除外卵黄样黄斑变性  老年黄斑变性早期 !左黄斑病变AMD瘢痕
# 双黄斑变性卵黄样黄斑变性  黄斑变性CNV  早期年龄相关性黄斑变性  !双黄斑区病变陈旧性不除外黄斑变性  黄斑变性早期  双黄斑病变湿性AMD 
# !双高度近视眼底病变黄斑病变  黄斑病变陈旧中浆

/usr/local/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


双青光眼!左继发视神经萎缩!右角膜白班眼底待查!左黄斑前膜                                           1
右视网膜中央静脉阻塞可能性大!右继发性黄斑前膜                                                 1
双老年性白内障!双视网膜动脉硬化!右玻璃体后脱离!右增殖性玻璃体视网膜病变原因待查!右继发黄斑前膜!左视网膜分支静脉阻塞!左黄斑囊样水肿    1
右玻璃体混浊机化!左视网膜前增殖膜!左继发黄斑前膜!双视网膜动脉硬化!双视网膜分支静脉阻塞陈旧!双底光凝术后!右屈光不正            1
左继发性黄斑前膜!双视网膜动脉硬化!右白内障摘除人工晶状体植入术后!左白内障                                  1
右视网膜分支静脉阻塞!右底光凝术后!右继发黄斑前膜!双视网膜动脉硬化!双屈光不正                                1
右继发性黄斑前膜!双高血压性视网膜病变                                                     1
双视网膜动脉硬化!双糖尿病视网膜病变NPDR2-3期!右继发黄斑前膜!双屈光不正                                1
左玻璃体积血!左视网膜分支静脉阻塞上方陈旧!左增殖性玻璃体视网膜病变!左继发黄斑前膜!双视网膜动脉硬化!右屈光不正               1
左陈旧性分支静阻!左继发性黄斑前膜                                                       1
双青光眼晚期继发视神经萎缩!左黄斑前膜                                                     1
双翼状胬肉!双老年性白内障!双视网膜动脉硬化!右视网膜分支静脉阻塞!右继发黄斑前膜!双黄斑陈旧病变!双屈光不正!左底出血原因待查        1
双RP原发继发!双黄斑前膜!右视网膜脱离待排                                                  1
右继发性黄斑前膜性质待定                          

继发黄斑裂孔  左黄斑板层裂孔! 黄斑裂孔I期!  右特发性全层黄斑裂孔  右黄斑裂孔继发视网膜脱离  右黄斑前膜!右假性黄斑裂孔  
双黄斑裂孔!右板层!左全层!左白内障术后  左可疑黄斑裂孔!左外伤后   右黄斑裂孔III期! 右黄斑裂孔IV期  右黄斑裂孔Ia期先兆裂孔!

左黄斑板层裂孔   双黄斑板层孔  双黄斑板层孔  左板层裂孔  黄斑板层孔 右黄斑可疑板层孔  右黄斑病变黄斑初期板层孔  
左黄斑板层孔 右黄斑区板层裂孔  右可疑板层裂孔!  双黄斑板层孔  右黄斑区板层裂孔   左黄斑病变板层孔  双黄斑板层孔  板层裂孔

In [25]:
# 黄斑裂孔_板层黄斑裂孔
def lk_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('板层|继发|特发|全层|假性裂孔|假孔|假性黄斑裂孔').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑板层裂孔', '黄斑裂孔_板层', str_i)
            str_i_1 = single_replace_fuc('黄斑初期板层孔', '黄斑裂孔_板层', str_i_0) 
            str_i_2 = single_replace_fuc('黄斑区板层裂孔', '黄斑裂孔_板层', str_i_1)
            str_i_3 = single_replace_fuc('黄斑病变板层孔', '黄斑裂孔_板层', str_i_2) 
            str_i_4 = single_replace_fuc('黄斑可疑板层孔', '黄斑裂孔_板层', str_i_3) 
            str_i_5 = single_replace_fuc('黄斑板层孔', '黄斑裂孔_板层', str_i_4)
            str_i_6 = single_replace_fuc('板层裂孔', '黄斑裂孔_板层', str_i_5)
            str_i_7 = single_replace_fuc('特发性全层黄斑裂孔', '黄斑裂孔_特发性_全层', str_i_6)
            str_i_8 = single_replace_fuc('特发性黄斑裂孔', '黄斑裂孔_特发性', str_i_7)
            str_i_9 = single_replace_fuc('特发黄斑裂孔', '黄斑裂孔_特发性', str_i_8)
            str_i_10 = single_replace_fuc('继发性黄斑裂孔', '黄斑裂孔_继发性', str_i_9)
            str_i_11 = single_replace_fuc('继发黄斑裂孔', '黄斑裂孔_继发性', str_i_10)
            str_i_12 = single_replace_fuc('黄斑前膜板层孔', '黄斑前膜+黄斑裂孔_板层', str_i_11)
            str_i_13 = single_replace_fuc('黄斑假性裂孔', '黄斑裂孔_假性', str_i_12)
            str_i_14 = single_replace_fuc('黄斑裂孔假孔', '黄斑裂孔_假性', str_i_13)
            str_i_15 = single_replace_fuc('黄斑假孔', '黄斑裂孔_假性', str_i_14)
            str_i_16 = single_replace_fuc('黄斑前膜假性裂孔', '黄斑前膜+黄斑裂孔_假性', str_i_15)
            str_i_17 = single_replace_fuc('黄斑前膜黄斑假孔', '黄斑前膜+黄斑裂孔_假性', str_i_16)
            str_i_17 = single_replace_fuc('假性黄斑裂孔', '黄斑裂孔_假性', str_i_16)
            df.at[i,'阅_gai'] = single_replace_fuc('黄斑前膜假孔', '黄斑前膜+黄斑裂孔_假性', str_i_17)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

In [26]:
lk_fuc(df_amd)

In [27]:
df_amd[df_amd['阅_gai'].str.contains('黄斑(.*)出血',re.I)]['阅_gai'].value_counts()  

左黄斑出血                                                                      33
右黄斑出血                                                                      25
左黄斑出血!双屈光不正                                                                 7
双高度近视!左黄斑出血                                                                 6
右黄斑出血!双屈光不正                                                                 6
双屈光不正!右黄斑出血                                                                 6
双高度近视!右黄斑出血                                                                 5
右黄斑病变出血                                                                     4
双屈光不正!左黄斑出血                                                                 4
双黄斑区图像暗!双可见处未见明显出血及渗出                                                       4
左黄斑区图像暗!双可见处未见明显出血及渗出                                                       4
右黄斑出血原因                                                                     3
双高血压视网膜病变!双黄斑病变!左底出血                                            

黄斑前膜裂孔
黄斑前膜继发性
黄斑前膜早期
继发性黄斑前膜
双黄斑前膜!双黄斑裂孔
右黄斑裂孔!右黄斑前膜
局灶性黄斑前膜 
黄斑变性黄斑前膜

In [28]:
# 黄斑前膜
def qm_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑前膜|黄斑病变前膜').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑前膜黄斑裂孔', '黄斑前膜+黄斑裂孔', str_i)
            str_i_1 = single_replace_fuc('黄斑裂孔黄斑前膜', '黄斑前膜+黄斑裂孔', str_i_0)
            str_i_2 = single_replace_fuc('黄斑前膜伴裂孔', '黄斑前膜+黄斑裂孔', str_i_1)
            str_i_3 = single_replace_fuc('黄斑前膜裂孔', '黄斑前膜+黄斑裂孔', str_i_2) 
            str_i_4 = single_replace_fuc('黄斑前膜黄斑裂洞', '黄斑前膜+黄斑裂孔', str_i_3) 
            str_i_5 = single_replace_fuc('黄斑前膜黄斑孔', '黄斑前膜+黄斑裂孔', str_i_4) 
            str_i_6 = single_replace_fuc('黄斑前膜可疑黄斑裂孔', '黄斑前膜+黄斑裂孔', str_i_5) 
            str_i_7 = single_replace_fuc('黄斑前膜早期', '黄斑前膜_早期', str_i_6) 
            str_i_8 = single_replace_fuc('黄斑前膜初期', '黄斑前膜_早期', str_i_7)
            str_i_9 = single_replace_fuc('黄斑前膜中期', '黄斑前膜_中期', str_i_8) 
            str_i_10 = single_replace_fuc('黄斑前膜晚期', '黄斑前膜_晚期', str_i_9)          
            str_i_11 = single_replace_fuc('黄斑前膜继发性', '黄斑前膜_继发性', str_i_10)
            str_i_12 = single_replace_fuc('黄斑前膜继发', '黄斑前膜_继发性', str_i_11)
            str_i_13 = single_replace_fuc('继发性黄斑前膜', '黄斑前膜_继发性', str_i_12)
            str_i_14 = single_replace_fuc('继发黄斑前膜', '黄斑前膜_继发性', str_i_13)
            str_i_15 = single_replace_fuc('黄斑前膜局灶性', '黄斑前膜_局灶性', str_i_14)
            str_i_16 = single_replace_fuc('黄斑前膜局灶', '黄斑前膜_局灶性', str_i_15)
            str_i_17 = single_replace_fuc('黄斑病变前膜', '黄斑前膜', str_i_16)
            df.at[i,'阅_gai'] = single_replace_fuc('局灶性黄斑前膜', '黄斑前膜_局灶性', str_i_17)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

In [29]:
qm_fuc(df_amd)

In [30]:
# 黄斑出血
def hbcx_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑(.*)出血').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑病变出血', '黄斑出血', str_i)
            str_i_1 = single_replace_fuc('黄斑变性出血', '黄斑出血', str_i_0)
            str_i_2 = single_replace_fuc('黄斑区网膜下出血', '黄斑出血', str_i_1) 
            str_i_3 = single_replace_fuc('黄斑区视网膜出血', '黄斑出血', str_i_2) 
            str_i_4 = single_replace_fuc('黄斑前视网膜出血', '黄斑出血', str_i_3)
            str_i_5 = single_replace_fuc('黄斑陈旧性病变出血', '黄斑出血', str_i_4) 
            str_i_6 = single_replace_fuc('黄斑区出血', '黄斑出血', str_i_5)
            str_i_7 = single_replace_fuc('黄斑部出血', '黄斑出血', str_i_6) 
            str_i_8 = single_replace_fuc('黄斑下出血', '黄斑出血', str_i_7) 
            str_i_9 = single_replace_fuc('黄斑上方出血', '黄斑出血', str_i_8) 
            str_i_10 = single_replace_fuc('黄斑鼻下出血', '黄斑出血', str_i_9) 
            str_i_11 = single_replace_fuc('黄斑点状出血', '黄斑出血', str_i_10) 
            str_i_12 = single_replace_fuc('黄斑颞上出血', '黄斑出血', str_i_11)
            str_i_13 = single_replace_fuc('黄斑区点状出血', '黄斑出血', str_i_12)
            str_i_14 = single_replace_fuc('黄斑颞下可疑出血', '黄斑出血', str_i_13)
            str_i_15 = single_replace_fuc('黄斑颞上可疑出血', '黄斑出血', str_i_14)
            str_i_16 = single_replace_fuc('黄斑鼻下可疑出血', '黄斑出血', str_i_15)
            str_i_17 = single_replace_fuc('黄斑区颞上方出血', '黄斑出血', str_i_16)
            str_i_18 = single_replace_fuc('黄斑前膜黄斑出血', '黄斑前膜+黄斑出血', str_i_17)
            df.at[i,'阅_gai']  = single_replace_fuc('黄斑前出血', '黄斑出血', str_i_18)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

In [31]:
hbcx_fuc(df_amd)

In [32]:
# 玻璃体黄斑牵拉综合征
def hbql_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑(.*)牵拉|玻璃体视网膜牵拉改变').findall(str_i))>0:
            str_i_0 = single_replace_fuc('黄斑玻璃体牵拉', '黄斑牵拉综合征', str_i)
            str_i_1 = single_replace_fuc('黄斑玻璃体牵拉综合征', '黄斑牵拉综合征', str_i_0)
            str_i_2 = single_replace_fuc('玻璃体黄斑牵拉综合征', '黄斑牵拉综合征', str_i_1) 
            str_i_3 = single_replace_fuc('黄斑区可疑玻璃体牵拉', '黄斑牵拉综合征', str_i_2) 
            str_i_4 = single_replace_fuc('玻璃体黄斑牵拉', '黄斑牵拉综合征', str_i_3)
            df.at[i,'阅_gai'] = single_replace_fuc('玻璃体视网膜牵拉改变', '黄斑牵拉综合征', str_i_4)
        elif len(re.compile('黄斑牵拉').findall(str_i))>0 and len(re.compile('黄斑牵拉综合征').findall(str_i))==0:
            print(str_i)
            df.at[i,'阅_gai'] = single_replace_fuc('黄斑牵拉', '黄斑牵拉综合征', str_i)
            print("__{}".format(df.at[i,'阅_gai']))
#             str_i_7 = single_replace_fuc('黄斑部出血', '黄斑牵拉综合征', str_i_6) 
#             str_i_8 = single_replace_fuc('黄斑下出血', '黄斑出血', str_i_7) 
#             str_i_9 = single_replace_fuc('黄斑上方出血', '黄斑出血', str_i_8) 
            
#             df.at[i,'阅_gai']  = single_replace_fuc('黄斑前出血', '黄斑出血', str_i_6)

        else:
            pass

In [33]:
hbql_fuc(df_amd)

In [34]:
df_amd[df_amd['阅_gai'].str.contains('视锥细胞',re.I)]['阅_gai'].value_counts()  

双黄斑微小视锥细胞损伤                  1
双屈光不正待除外视锥细胞营养不良             1
双先天性黄斑变性Stargadt病视锥细胞营养不良    1
双屈光不正!双待除外视锥细胞营养不良           1
双先天性黄斑营养不良视锥细胞营养不良           1
双先天性黄斑营养不良视锥细胞营养不良可能性大       1
Name: 阅_gai, dtype: int64

In [35]:
# 中浆、中渗
def zs_and_zj_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('中心性渗出性脉络膜视网膜病变|Rieger中心性视网膜炎|青年性出血性黄斑病变|中心性浆液性脉络膜视网膜病变|黄斑病变中浆|黄斑病变中渗').findall(str_i))>0:
            str_i_0 = single_replace_fuc('中心性渗出性脉络膜视网膜病变', '黄斑病变_中渗', str_i)
            str_i_1 = single_replace_fuc('Rieger中心性视网膜炎', '黄斑病变_中渗_Rieger中心性视网膜炎', str_i_0)
            str_i_2 = single_replace_fuc('青年性出血性黄斑病变', '黄斑病变_中渗_青年性出血性黄斑病变', str_i_1) 
            str_i_3 = single_replace_fuc('黄斑病变中渗', '黄斑病变_中渗', str_i_2)
            str_i_4 = single_replace_fuc('黄斑病变中浆', '黄斑病变_中浆', str_i_3) 
            df.at[i,'阅_gai'] = single_replace_fuc('中心性浆液性脉络膜视网膜病变', '黄斑病变_中浆', str_i_4) 
        elif len(re.compile('中渗').findall(str_i))>0 and len(re.compile('黄斑病变_中渗').findall(str_i))==0:
            print(str_i)
            df.at[i,'阅_gai'] = single_replace_fuc('中渗', '黄斑病变_中渗', str_i)
            print("__{}".format(df.at[i,'阅_gai']))
        elif len(re.compile('中浆').findall(str_i))>0 and len(re.compile('黄斑病变_中浆').findall(str_i))==0:
            print(str_i)
            df.at[i,'阅_gai'] = single_replace_fuc('中浆', '黄斑病变_中浆', str_i)
            print("__{}".format(df.at[i,'阅_gai']))
            
#             df.at[i,'阅_gai']  = single_replace_fuc('黄斑前出血', '黄斑出血', str_i_6)

        else:
            pass

In [36]:
zs_and_zj_fuc(df_amd)

左中渗病
__左黄斑病变_中渗病
右可疑中浆病
__右可疑黄斑病变_中浆病
左中浆
__左黄斑病变_中浆
左陈旧中浆
__左陈旧黄斑病变_中浆
右中浆!双屈光不正
__右黄斑病变_中浆!双屈光不正
右中浆!左可疑青光眼
__右黄斑病变_中浆!左可疑青光眼
右中浆!左视网膜脉络膜病变
__右黄斑病变_中浆!左视网膜脉络膜病变
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右慢性中浆!双高血压视网膜病变
__右慢性黄斑病变_中浆!双高血压视网膜病变
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右陈旧中浆!左中浆复发可能性大!双屈光不正
__右陈旧黄斑病变_中浆!左黄斑病变_中浆复发可能性大!双屈光不正
右中渗
__右黄斑病变_中渗
右中浆待查
__右黄斑病变_中浆待查
右中浆
__右黄斑病变_中浆
双中浆病
__双黄斑病变_中浆病
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左陈旧性黄斑病变性质待定中渗!双屈光不正!双老视
__左陈旧性黄斑病变性质待定黄斑病变_中渗!双屈光不正!双老视
右中浆!双底动脉硬化
__右黄斑病变_中浆!双底动脉硬化
左中浆
__左黄斑病变_中浆
右中浆可能性大
__右黄斑病变_中浆可能性大
右中浆
__右黄斑病变_中浆
右中浆!双视网膜出血高血压性
__右黄斑病变_中浆!双视网膜出血高血压性
右中浆
__右黄斑病变_中浆
左中浆待排
__左黄斑病变_中浆待排
右中浆!双屈光不正
__右黄斑病变_中浆!双屈光不正
右中浆
__右黄斑病变_中浆
双黄斑病变!左中浆
__双黄斑病变!左黄斑病变_中浆
右考虑中浆
__右考虑黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆高血压视网膜病变
__右黄斑病变_中浆高血压视网膜病变
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆!双糖尿病视网膜病变
__左黄斑病变_中浆!双糖尿病视网膜病变
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
双中浆
__双黄斑病变_中浆
左黄斑病变以中浆病或特发

左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆!左先天白内障
__右黄斑病变_中浆!左先天白内障
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
双中浆小柳原田综合症
__双黄斑病变_中浆小柳原田综合症
左考虑中浆
__左考虑黄斑病变_中浆
左结膜炎!左中浆不除外
__左结膜炎!左黄斑病变_中浆不除外
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右陈旧黄斑病变!左中浆
__右陈旧黄斑病变!左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆待查
__右黄斑病变_中浆待查
左中浆
__左黄斑病变_中浆
右陈旧中浆
__右陈旧黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆可能性大
__左黄斑病变_中浆可能性大
右中浆
__右黄斑病变_中浆
左中浆可能性大
__左黄斑病变_中浆可能性大
双中浆
__双黄斑病变_中浆
左中浆可能
__左黄斑病变_中浆可能
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆!双糖尿病视网膜病变I期
__左黄斑病变_中浆!双糖尿病视网膜病变I期
右中浆
__右黄斑病变_中浆
左老年性中浆
__左老年性黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右屈光不正!左中浆
__右屈光不正!左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中渗
__左黄斑病变_中渗
左中浆
__左黄斑病变_中浆
右中浆病可能性大
__右黄斑病变_中浆病可能性大
右中浆
__右黄斑病变_中浆
双黄斑陈旧病变待除外中浆病
__双黄斑陈旧病变待除外黄斑病变_中浆病
左中浆
__左黄斑病变_中浆
右黄斑病变陈旧中浆
__右黄斑病变陈旧黄斑病变_中浆
双中浆
__双黄斑病变_中浆
右中浆!双屈光不正
__右黄斑病变_中浆!双屈光不正
左中

左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右黄斑病变性质待查中浆老年性黄斑变性!双视网膜动脉硬化!双屈光不正
__右黄斑病变性质待查黄斑病变_中浆老年性黄斑变性!双视网膜动脉硬化!双屈光不正
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
双慢性中浆不除外
__双慢性黄斑病变_中浆不除外
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右黄斑水肿中浆待排
__右黄斑水肿黄斑病变_中浆待排
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
双中浆!双Vogt-小柳-原田综合征待除外!右屈光不正
__双黄斑病变_中浆!双Vogt-小柳-原田综合征待除外!右屈光不正
左中浆
__左黄斑病变_中浆
左黄斑病变性质CNV中浆
__左黄斑病变性质CNV黄斑病变_中浆
右陈旧中浆
__右陈旧黄斑病变_中浆
右中浆!右脉络膜色素痣!双屈光不正
__右黄斑病变_中浆!右脉络膜色素痣!双屈光不正
左中浆可能性大
__左黄斑病变_中浆可能性大
左黄斑病变陈旧中浆
__左黄斑病变陈旧黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左黄斑病变CSC中渗
__左黄斑病变CSC黄斑病变_中渗
左中浆
__左黄斑病变_中浆
双有眼中浆
__双有眼黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
双中浆
__双黄斑病变_中浆
左中浆
__左黄斑病变_中浆
右中渗病
__右黄斑病变_中渗病
左中渗病
__左黄斑病变_中渗病
右中浆
__右黄斑病变_中浆
双屈光不正!右中浆
__双屈光不正!右黄斑病变_中浆
右中浆!左黄斑陈旧病变
__右黄斑病变_中浆!左黄斑陈旧病变
左中浆
__左黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
右中浆
__右黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆
__左黄斑病变_中浆
左中浆!双屈光不正
__左黄斑病变_中浆!双屈光不正
右白内障!左视网膜病变脱离中浆
__

视网膜色素变性。和黄斑病变不一样。视网膜色素变性是遗传性眼病，眼底有骨细胞沉着，视网膜颜色污浊，有夜盲。黄斑病变原因复杂疾病种类多样，眼病视力低下

In [37]:
df_amd[df_amd['阅_gai'].str.contains('ME',re.I)]['阅_gai'].value_counts() #CSME：有临床意义的黄斑水肿  # CSME：有临床意义的黄斑水肿

右BRVOME                                    10
双DR!双DME                                    6
左BRVOME                                     6
双糖尿病视网膜病变!双DME                              5
双PDR!双DME                                   4
右CRVOME                                     4
左CRVOME                                     4
右CRVO+CME                                   3
双DRIII-IV!双DME                              3
双DRPRP后!双DME                                3
左CRVOCME                                    2
左BRVO+CME                                   2
右BRVO!右CME                                  2
双DR激光后!双DME                                 2
右玻璃体积血!左NPDR!左DME!左高血压视网膜病变                 2
左CRVO!左CME                                  2
右BRVO+CME                                   2
左BRVO!左CME                                  2
双糖网!双DME                                    2
右黄斑前膜黄斑水肿!左RVOME                            1
双NPDR中度!右DME!双白内障                           1
双重度非增生性糖尿病视网膜病变!双DME              

#### 黄斑囊样水肿(cystoid macular edema，CME)是常见眼底病，但它不是一独立的疾病，而是很多眼底疾病在黄斑区的表现。
引起黄斑囊样水肿的最常见的疾病有：视网膜静脉阻塞、糖尿病性视网膜病变、视网膜血管炎、黄斑区视网膜前膜、视网膜毛细血管扩张症或Coats病、葡萄膜炎、中间葡萄膜炎、白内障或其他内眼术后、黄斑区脉络膜的新生血管、视网膜色素变性等。
#### 黄斑水肿和黄斑囊样水肿的区别？
黄斑囊样水肿是在黄斑水肿的基础上，由于黄斑区的视网膜神经上皮层的液体堆积并挤推黄斑纤维，形成囊样水肿。黄斑囊样水肿，它不是一种独立的病，而是一个临床的体征，可以发生在很多眼底的疾病中。它发生的原因主要是眼底视网膜内屏障，即血视网膜屏障受到破坏，多种血管里的液体、蛋白渗出来以后造成组织的水肿。常见的有老年性黄斑变性、糖尿病、眼底出血、静脉阻塞、血管炎、高血压、眼底出血等很多的病。此外，一些全身病也可以造成黄斑的水肿。所以引起黄斑囊样水肿的原因很多，治疗上也比较复杂，根据病情、根据它的原发病，首先进行治疗。

In [38]:
# 黄斑水肿——黄斑囊样水肿  左黄斑水肿!左黄斑前膜   双黄斑前膜!双黄斑水肿
def hbsz_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑(.*)水肿',re.I).findall(str_i))>0:
            str_i_0 = single_replace_fuc('黄斑病变前膜水肿', '黄斑病变_黄斑水肿+黄斑病变_黄斑前膜', str_i)
            str_i_1 = single_replace_fuc('黄斑囊样水肿', '黄斑水肿_黄斑囊样水肿', str_i_0)
            str_i_2 = single_replace_fuc('黄斑水肿前膜', '黄斑病变_黄斑水肿+黄斑病变_黄斑前膜', str_i_1) 
            str_i_3 = single_replace_fuc('黄斑病变水肿', '黄斑水肿', str_i_2) 
            str_i_4 = single_replace_fuc('黄斑前膜黄斑水肿', '黄斑病变_黄斑水肿+黄斑病变_黄斑前膜', str_i_3) 
            str_i_5 = single_replace_fuc('黄斑区水肿', '黄斑水肿', str_i_4)
            str_i_6 = single_replace_fuc('黄斑钝伤性水肿', '黄斑水肿', str_i_5)
            str_i_7 = single_replace_fuc('黄斑区视网膜水肿', '黄斑水肿', str_i_6)
            str_i_8 = single_replace_fuc('黄斑区视网膜轻度水肿', '黄斑水肿', str_i_7)
            str_i_9 = single_replace_fuc('黄斑牵拉黄斑水肿', '黄斑病变_黄斑牵拉综合征+黄斑病变_黄斑水肿', str_i_8)
            str_i_10 = single_replace_fuc('黄斑病变性质黄斑水肿', '黄斑水肿', str_i_9)
            str_i_11 = single_replace_fuc('黄斑前膜黄斑囊样水肿', '黄斑病变_黄斑前膜+黄斑病变_黄斑水肿_黄斑囊样水肿', str_i_10)
            str_i_12 = single_replace_fuc('黄斑区可疑水肿', '黄斑水肿', str_i_11)
            str_i_13 = single_replace_fuc('糖尿病性黄斑水肿', '黄斑水肿_糖尿病性', str_i_12)
            str_i_14 = single_replace_fuc('黄斑水肿糖尿病性', '黄斑水肿_糖尿病性', str_i_13)
            str_i_15 = single_replace_fuc('重度糖尿病性黄斑水肿', '黄斑水肿_糖尿病性_重度', str_i_14)
            str_i_16 = single_replace_fuc('中度糖尿病性黄斑水肿', '黄斑水肿_糖尿病性_中度', str_i_15)
            df.at[i,'阅_gai'] = single_replace_fuc('轻度糖尿病性黄斑水肿', '黄斑水肿_糖尿病性_轻度', str_i_16)
        elif len(re.compile('BRVOME|BRVO+CME|BRVO合并CME|BRVO+ME|BRVO-ME',re.I).findall(str_i))>0:
            print(str_i)
            str_j_0  = single_replace_fuc('BRVO合并CME', '视网膜分支静脉阻塞+黄斑水肿_黄斑囊样水肿', str_i)
            str_j_1  = single_replace_fuc('BRVO+CME', '视网膜分支静脉阻塞+黄斑水肿_黄斑囊样水肿', str_j_0)
            str_j_2  = single_replace_fuc('BRVO+ME', '视网膜分支静脉阻塞+黄斑水肿', str_j_1)
            str_j_3  = single_replace_fuc('BRVO-ME', '视网膜分支静脉阻塞+黄斑水肿', str_j_2)
            df.at[i,'阅_gai'] = single_replace_fuc('BRVOME', '视网膜分支静脉阻塞+黄斑水肿', str_j_3)
            print("__{}".format(df.at[i,'阅_gai']))
        elif len(re.compile('CRVOME|CRVO+CME|CRVOCME|CRVO合并CME|CRVO+ME|CRVO-ME',re.I).findall(str_i))>0:
            print(str_i)
            str_k_0  = single_replace_fuc('CRVOCME', '视网膜中央静脉阻塞+黄斑水肿_黄斑囊样水肿', str_i)
            str_k_1  = single_replace_fuc('CRVOME', '视网膜中央静脉阻塞+黄斑水肿', str_k_0)
            str_k_2  = single_replace_fuc('CRVO+ME', '视网膜中央静脉阻塞+黄斑水肿', str_k_1)
            str_k_3  = single_replace_fuc('CRVO-ME', '视网膜中央静脉阻塞+黄斑水肿', str_k_2)
            str_k_4  = single_replace_fuc('CRVO合并CME', '视网膜中央静脉阻塞+黄斑水肿_黄斑囊样水肿', str_k_3)
            df.at[i,'阅_gai']  = single_replace_fuc('CRVO+CME', '视网膜中央静脉阻塞+黄斑水肿_黄斑囊样水肿', str_k_4)
            print("__{}".format(df.at[i,'阅_gai']))
        elif len(re.compile('CSME|ERMME',re.I).findall(str_i))>0:
            print(str_i)
            str_l_0 = single_replace_fuc('CSME', '黄斑水肿', str_i)
            df.at[i,'阅_gai']  = single_replace_fuc('ERMME', '黄斑水肿', str_l_0)   
            print("__{}".format(df.at[i,'阅_gai']))
        elif len(re.compile('DME|CME',re.I).findall(str_i))>0:
            print(str_i)
            str_m_0 = single_replace_fuc('DME', '黄斑水肿_糖尿病性', str_i)
            df.at[i,'阅_gai']  = single_replace_fuc('CME', '黄斑水肿_黄斑囊样水肿', str_m_0)
            print("__{}".format(df.at[i,'阅_gai']))            
        else:
            pass

In [39]:
hbsz_fuc(df_amd)

左CRVO!左CME
__左CRVO!左黄斑水肿_黄斑囊样水肿
左BRVO+CME!左开青待排
__左BRVO+黄斑水肿_黄斑囊样水肿!左开青待排
双DRII-III!双DME
__双DRII-III!双黄斑水肿_糖尿病性
双NPDR中-重!双黄斑前膜!双DME
__双NPDR中-重!双黄斑前膜!双黄斑水肿_糖尿病性
双糖尿病视网膜病变!双DME
__双糖尿病视网膜病变!双黄斑水肿_糖尿病性
右BRVO合并CME
__右视网膜分支静脉阻塞+黄斑水肿_黄斑囊样水肿
双NPDR中-重度!双DME高血压视网膜病变II
__双NPDR中-重度!双黄斑水肿_糖尿病性高血压视网膜病变II
右葡萄膜炎!右CME
__右葡萄膜炎!右黄斑水肿_黄斑囊样水肿
双DR!双DME
__双DR!双黄斑水肿_糖尿病性
双中度非增生性糖尿病视网膜病变!双DME
__双中度非增生性糖尿病视网膜病变!双黄斑水肿_糖尿病性
左BRVOME
__左视网膜分支静脉阻塞+黄斑水肿
双DR!双DME!左重
__双DR!双黄斑水肿_糖尿病性!左重
右陈旧BRVO!左DME
__右陈旧BRVO!左黄斑水肿_糖尿病性
双PDR!右PRP后!左底待查!右DME
__双PDR!右PRP后!左底待查!右黄斑水肿_糖尿病性
双糖尿病视网膜病变!双DME
__双糖尿病视网膜病变!双黄斑水肿_糖尿病性
双糖尿病视网病变!双DME
__双糖尿病视网病变!双黄斑水肿_糖尿病性
双糖尿病视网膜病变!右DME
__双糖尿病视网膜病变!右黄斑水肿_糖尿病性
右CRVO!右CME
__右CRVO!右黄斑水肿_黄斑囊样水肿
右BRVOME!左高度近视眼底改变
__右视网膜分支静脉阻塞+黄斑水肿!左高度近视眼底改变
双DRPRP后!双DME
__双DRPRP后!双黄斑水肿_糖尿病性
双白内障!双糖尿病视网膜病变!双DME
__双白内障!双糖尿病视网膜病变!双黄斑水肿_糖尿病性
右CRVO+CME
__右CRVO+黄斑水肿_黄斑囊样水肿
右BRVOME
__右视网膜分支静脉阻塞+黄斑水肿
双DR激光后!双DME
__双DR激光后!双黄斑水肿_糖尿病性
左PDR!右DR!双DME待查!双IOL眼
__左PDR!右DR!双黄斑水肿_糖尿病性待查!双IOL眼
双BDR!左DME
__双BDR!左黄斑水肿_糖尿病性
右CRVO+CME
__右CRVO+

In [40]:
# 黄斑病变营养不良性
def hbyybl_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑(.*)营养不良').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑病变营养不良性', '黄斑病变_营养不良', str_i)
            str_i_1 = single_replace_fuc('黄斑病变营养不良', '黄斑病变_营养不良', str_i_0)
            str_i_2 = single_replace_fuc('黄斑营养不良', '黄斑病变_营养不良', str_i_1) 
            str_i_3 = single_replace_fuc('黄斑陈旧病变营养不良', '黄斑病变_营养不良', str_i_2) 
            str_i_4 = single_replace_fuc('黄斑病变萎缩性营养不良', '黄斑病变_营养不良_萎缩性', str_i_3) 
            str_i_5 = single_replace_fuc('黄斑萎缩性病变营养不良', '黄斑病变_营养不良_萎缩性', str_i_4)
            str_i_6 = single_replace_fuc('黄斑病变性质营养不良', '黄斑病变_营养不良', str_i_5)
            str_i_7 = single_replace_fuc('黄斑病变变性或营养不良', '黄斑病变_黄斑变性+黄斑病变_营养不良', str_i_6)
            str_i_8 = single_replace_fuc('遗传性黄斑营养不良', '黄斑病变_营养不良_先天性', str_i_7)
            str_i_9 = single_replace_fuc('黄斑病变变性营养不良', '黄斑病变_营养不良', str_i_8)
            df.at[i,'阅_gai'] = single_replace_fuc('先天性黄斑营养不良', '黄斑病变_营养不良_先天性', str_i_9)
#         elif len(re.compile('黄斑牵拉').findall(str_i))>0 and len(re.compile('黄斑病变_营养不良').findall(str_i))==0:
#             df.at[i,'阅_gai']  = single_replace_fuc('黄斑前出血', '黄斑出血', str_i_6)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

In [41]:
hbyybl_fuc(df_amd)

In [42]:
# 黄斑区色素上皮改变、脱离
def hbssspgb_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑区色素上皮').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑区色素上皮病变', '黄斑病变_黄斑区色素上皮改变', str_i)
            str_i_1 = single_replace_fuc('黄斑区色素上皮改变', '黄斑病变_黄斑区色素上皮改变', str_i_0)
            str_i_2 = single_replace_fuc('黄斑区色素上皮脱离', '黄斑病变_黄斑区色素上皮脱离', str_i_1) 
            df.at[i,'阅_gai'] = single_replace_fuc('黄斑区色素上皮层脱离', '黄斑病变_黄斑区色素上皮脱离', str_i_2) 
#             str_i_4 = single_replace_fuc('黄斑区色素上皮层脱离', '黄斑病变_营养不良_萎缩性', str_i_3) 
#             str_i_5 = single_replace_fuc('黄斑萎缩性病变营养不良', '黄斑病变_营养不良_萎缩性', str_i_4)
#             df.at[i,'阅_gai'] = single_replace_fuc('先天性黄斑营养不良', '黄斑病变_营养不良_先天性', str_i_9)
#         elif len(re.compile('黄斑牵拉').findall(str_i))>0 and len(re.compile('黄斑病变_营养不良').findall(str_i))==0:
#             df.at[i,'阅_gai']  = single_replace_fuc('黄斑前出血', '黄斑出血', str_i_6)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

In [43]:
hbssspgb_fuc(df_amd)  # 黄斑区色素上皮改变、脱离

### 黄斑部囊样变性
黄斑部囊样变性最常见的原因为外伤,尤其是眼球挫伤后,在黄斑部出现水肿或出血,以致形成囊样变性,也可由于老年黄斑变性、变性性近视、眼底后极部炎症或血管性疾病引起的水肿所致。

In [44]:
# 黄斑部囊样变性
def hbnybx_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑(.*)囊样变性').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('黄斑部囊样变性', '黄斑病变_黄斑囊样变性', str_i)
            df.at[i,'阅_gai'] = single_replace_fuc('黄斑囊样变性', '黄斑病变_黄斑囊样变性', str_i_0)
#             df.at[i,'阅_gai'] = single_replace_fuc('黄斑区色素上皮层脱离', '黄斑病变_黄斑区色素上皮脱离', str_i_2) 
        else:
            pass

In [45]:
hbnybx_fuc(df_amd)

In [46]:
#  黄斑变性AMD——'早期(.*)'——顺序调整为——(.*)早期——名称统一 
def hz_shunxu_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('AMD可能性大|AMD可能大|年龄相关性黄斑变|干性黄斑变性|湿性黄斑变性|湿性AMD|干性AMD|干性老年性黄斑变性|湿性老年性黄斑变性|(.)期黄斑变性|年龄相关性黄斑变性|老年黄斑病变|老年黄斑变性|老年性黄斑病变|(.)期老年性黄斑变性').findall(str_i))>0:
#             print(str_i)
            str_i_0 = single_replace_fuc('年龄相关性黄斑变性', '老年性黄斑变性', str_i)
            str_i_1 = single_replace_fuc('老年黄斑病变', '老年性黄斑变性', str_i_0) 
            str_i_2 = single_replace_fuc('老年黄斑变性', '老年性黄斑变性', str_i_1)
            str_i_3 = single_replace_fuc('老年性黄斑病变', '老年性黄斑变性', str_i_2)
            str_i_4 = single_replace_fuc('早期老年性黄斑变性', '老年性黄斑变性早期', str_i_3)
            str_i_5 = single_replace_fuc('中期老年性黄斑变性', '老年性黄斑变性中期', str_i_4)
            str_i_6 = single_replace_fuc('晚期老年性黄斑变性', '老年性黄斑变性晚期', str_i_5)
            str_i_7 = single_replace_fuc('早期黄斑变性', '黄斑变性早期', str_i_6)
            str_i_8 = single_replace_fuc('中期黄斑变性', '黄斑变性中期', str_i_7)
            str_i_9 = single_replace_fuc('湿性老年性黄斑变性', '老年性黄斑变性湿性', str_i_8)
            str_i_10 = single_replace_fuc('干性老年性黄斑变性', '老年性黄斑变性干性', str_i_9)
            str_i_11 = single_replace_fuc('干性AMD', 'AMD干性', str_i_10)
            str_i_12 = single_replace_fuc('湿性AMD', 'AMD湿性', str_i_11)
            str_i_13 = single_replace_fuc('湿性黄斑变性', '老年性黄斑变性湿性', str_i_12)
            str_i_14 = single_replace_fuc('干性黄斑变性', '老年性黄斑变性干性', str_i_13)
            str_i_15 = single_replace_fuc('晚期黄斑变性', '黄斑变性晚期', str_i_14)
            str_i_16 = single_replace_fuc('AMD可能大', 'AMD', str_i_15)
            str_i_17 = single_replace_fuc('AMD可能性大', 'AMD', str_i_16)
            df.at[i,'阅_gai'] = single_replace_fuc('年龄相关性黄斑变', '老年性黄斑变性', str_i_17)
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

    check_df_sh = df[df['阅_gai'].str.contains('AMD可能性大|AMD可能大|年龄相关性黄斑变|干性黄斑变性|湿性黄斑变性|湿性AMD|干性AMD|干性老年性黄斑变性|湿性老年性黄斑变性|(.)期黄斑变性|年龄相关性黄斑变性|老年黄斑病变|老年黄斑变性|老年性黄斑病变|(.)期老年性黄斑变性')]
    if check_df_sh.shape[0] > 0:
        print("剩余{}条_早期_湿性——黄斑病变_数据未转化".format(check_df_sh.shape[0]))
        print(check_df_sh['阅_gai'])
        hz_shunxu_fuc(df)
    else:
        print('转化结束！！！')

In [47]:
hz_shunxu_fuc(df_amd)

转化结束！！！


In [48]:
#  黄斑变性AMD————名称统一
def hz_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('外伤黄斑病变|黄斑病变性质|黄斑陈旧性病变|陈旧性黄斑病变|陈旧黄斑病变|黄斑陈旧病变|黄斑病变中浆|黄斑病变高度近视|高度近视黄斑病变|黄斑病变外伤|CNV|中心性浆液性脉络膜视网膜病变|老年性黄斑变性|外伤性黄斑病变').findall(str_i))>0:
#             print(str_i)
            str_i_0  = single_replace_fuc('黄斑陈旧性病变', '黄斑病变', str_i) 
            str_i_1  = single_replace_fuc('陈旧性黄斑病变', '黄斑病变', str_i_0)
            str_i_2  = single_replace_fuc('黄斑病变性质', '黄斑病变', str_i_1)
            str_i_3  = single_replace_fuc('黄斑陈旧病变', '黄斑病变', str_i_2) 
            str_i_4  = single_replace_fuc('陈旧黄斑病变', '黄斑病变', str_i_3)
            str_i_5 = single_replace_fuc('外伤性黄斑病变', '黄斑病变_外伤性', str_i_4)
            str_i_6 = single_replace_fuc('外伤黄斑病变', '黄斑病变_外伤性', str_i_5) 
            str_i_7 = single_replace_fuc('黄斑病变外伤', '黄斑病变_外伤性', str_i_6)
            str_i_8 = single_replace_fuc('高度近视黄斑病变', '黄斑病变_高度近视', str_i_7)
            str_i_9 = single_replace_fuc('黄斑病变高度近视', '黄斑病变_高度近视', str_i_8)
            str_i_10  = single_replace_fuc('黄斑病变中浆', '黄斑病变_中浆', str_i_9) 
            str_i_11 = single_replace_fuc('CNV', '黄斑病变_黄斑变性_AMD_湿性', str_i_10)
            str_i_12 = single_replace_fuc('中心性浆液性脉络膜视网膜病变', '黄斑病变_中浆', str_i_11)
            df.at[i,'阅_gai'] = single_replace_fuc('老年性黄斑变性', '黄斑病变_黄斑变性_AMD', str_i_12) 
            
#             print("__{}".format(df.at[i,'阅_gai']))
        else:
            pass

    check_df_sh = df[df['阅_gai'].str.contains('外伤黄斑病变|黄斑病变性质|黄斑陈旧性病变|陈旧性黄斑病变|陈旧黄斑病变|黄斑陈旧病变|黄斑病变中浆|黄斑病变高度近视|高度近视黄斑病变|黄斑病变外伤|CNV|中心性浆液性脉络膜视网膜病变|外伤性黄斑病变')]
    if check_df_sh.shape[0] > 0:
        print("剩余{}条_黄斑病变_数据未转化".format(check_df_sh.shape[0]))
        print(check_df_sh['阅_gai'])
        hz_normal_fuc(df)
        
    else:
        print('转化结束！！！')

In [49]:
hz_normal_fuc(df_amd)    # 调用光凝术后的名称统一函数

剩余9条_黄斑病变_数据未转化
270                                     左黄斑病变性质待定!双屈光不正!双老视
24408     双病理性近视!右黄斑陈旧黄斑病变_黄斑变性_AMD_湿性!左黄斑出血黄斑病变_黄斑变性_AM...
29108                         右黄斑陈旧黄斑病变_黄斑变性_AMD_湿性病理性近视AMD
31511                   双病理性近视!双后巩膜葡萄肿!左陈旧性黄斑病变_黄斑变性_AMD_湿性
40468          双高度近视视网膜病变!右玻璃体后脱离!左黄斑病变陈旧黄斑病变_黄斑变性_AMD_湿性瘢痕
72593     双病理性近视待查!双黄斑萎缩性病变!左陈旧性黄斑病变_黄斑变性_AMD_湿性高血压视网膜病变...
100539                                    左玻璃体后脱离!双黄斑病变性质待查
119229               双高度近视视网膜改变!左黄斑瘢痕陈旧黄斑病变_黄斑变性_AMD_湿性可能性大
208641                                   双高度近视黄斑病变_黄斑变性_AMD
Name: 阅_gai, dtype: object
转化结束！！！


In [50]:
## 黄斑变性AMD——黄斑病变_二、三级标签名称的统一化
def amd_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        else:
            a  = 'Stargardt病'
            strinfo_0 = re.compile('Stargard',re.I)
            strinfo_1 = re.compile('Stargardt',re.I)  
            b  = '黄斑病变_黄斑变性_AMD_干性'
            strinfo_2 = re.compile('黄斑萎缩性病变|黄斑非渗出性')  #右甚——定义为“右中期”——“右”
            c  = '瘢痕期'
            strinfo_3 = re.compile('瘢痕')
            strinfo_4 = re.compile('瘢痕期')
            strinfo_5 = re.compile('角膜瘢痕')  
            d = '黄斑病变_黄斑前膜'
            strinfo_6 = re.compile('黄斑病变前膜')
            strinfo_7 = re.compile('黄斑病变前膜水肿')
            e = '黄斑病变_中浆'
            strinfo_8 = re.compile('中浆')
            strinfo_9 = re.compile('黄斑病变中浆')
            strinfo_10 = re.compile('黄斑病变_中浆')
            f  = 'Best病'
            strinfo_11 = re.compile('Best|BEST|卵黄样黄斑变性',re.I)
            g  = '瘢痕'
            strinfo_12 = re.compile('陈旧瘢痕|盘状瘢痕|部分瘢痕|大部分瘢痕|近瘢痕',re.I)
            
            if (len(strinfo_0.findall(str_i))>0) and (len(strinfo_1.findall(str_i))==0) :  # #替换仅为糖尿病视网膜病变的标签
#                 print(str_i)
                df.at[i,'阅_gai'] = strinfo_0.sub(a,str_i)
#                 print(df.at[i,'阅_gai'])
            elif (len(strinfo_2.findall(str_i))>0):
#                 print(str_i)
                df.at[i,'阅_gai'] = strinfo_2.sub(b,str_i)
#                 print('__'+df.at[i,'阅_gai'])
            elif (len(strinfo_3.findall(str_i))>0) and (len(strinfo_4.findall(str_i))==0):
                if len(strinfo_5.findall(str_i)) > 0:  # 无——角膜瘢痕
                    pass
                else:
#                     print(str_i)
                    df.at[i,'阅_gai'] = strinfo_3.sub(c,str_i)
#                     print('__'+df.at[i,'阅_gai'])
            elif (len(strinfo_6.findall(str_i))>0) and (len(strinfo_7.findall(str_i))==0):
#                 print(str_i)
                df.at[i,'阅_gai'] = strinfo_6.sub(d,str_i)
#                 print('__'+df.at[i,'阅_gai'])
            elif (len(strinfo_8.findall(str_i))>0) and (len(strinfo_9.findall(str_i))==0):
                if len(strinfo_10.findall(str_i)) > 0:  # 无——角膜瘢痕
                    pass
                else:
#                     print(str_i)
                    df.at[i,'阅_gai'] = strinfo_8.sub(e,str_i)
#                     print('__'+df.at[i,'阅_gai'])
            elif (len(strinfo_11.findall(str_i))>0):  # #替换仅为糖尿病视网膜病变的标签
#                 print(str_i)
                df.at[i,'阅_gai'] = strinfo_11.sub(f,str_i)
#                 print(df.at[i,'阅_gai'])
            elif (len(strinfo_12.findall(str_i))>0):  # #替换仅为糖尿病视网膜病变的标签
#                 print(str_i)
                df.at[i,'阅_gai'] = strinfo_12.sub(g,str_i)
#                 print(df.at[i,'阅_gai'])
            elif (len(re.compile('STARGADT病|STARGARDT病|stargardt病').findall(str_i))>0):  # #替换仅为糖尿病视网膜病变的标签
                print(str_i)
                df.at[i,'阅_gai'] = re.compile('STARGADT病|STARGARDT病|stargardt病').sub('Stargardt病',str_i)
                print(df.at[i,'阅_gai'])
            else:
                pass

    check_df_jb = df[df['阅_gai'].str.contains('BEST病|黄斑萎缩性病变|黄斑非渗出性|陈旧瘢痕|盘状瘢痕|部分瘢痕|大部分瘢痕|近瘢痕')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
    if check_df_jb.shape[0] > 0:
        print("剩余{}条数据未转化为——黄斑病变".format(check_df_jb.shape[0]))
        print(check_df_jb[['阅片诊断','阅_gai']])
        amd_normal_fuc(df)
    else:
        print('转化结束！！！')

In [51]:
amd_normal_fuc(df_amd)  

双黄斑病变待定黄斑前膜STARGARDT病
双黄斑病变待定黄斑前膜Stargardt病
双STARGARDT病
双Stargardt病
双黄斑变性STARGARDT病
双黄斑变性Stargardt病
双黄斑病变stargardt病
双黄斑病变Stargardt病
双黄斑水肿后妊娠高血压!双黄斑变性STARGADT病
双黄斑水肿后妊娠高血压!双黄斑变性Stargardt病
双STARGARDT病
双Stargardt病
双黄斑病变STARGARDT病合并!右CN
双黄斑病变Stargardt病合并!右CN
左待除外黄斑病变_营养不良STARGARDT病或锥杆细胞营养不良
左待除外黄斑病变_营养不良Stargardt病或锥杆细胞营养不良
双黄斑病变stargardt病
双黄斑病变Stargardt病
双不除外STARGARDT病
双不除外Stargardt病
双STARGARDT病
双Stargardt病
双STARGARDT病!双屈光不正!双老视
双Stargardt病!双屈光不正!双老视
剩余29条数据未转化为——黄斑病变
                                                  阅片诊断                                           阅_gai
2819                                 双眼湿性AMD(右眼部分瘢痕形成)                                 双AMD湿性!右部分瘢痕期形成
9024                                   左眼黄斑CNV(盘状瘢痕形成)                      左黄斑黄斑病变_黄斑变性_AMD_湿性盘状瘢痕期形成
17004                               双眼湿性AMD(大部分盘状瘢痕形成)                                双AMD湿性大部分盘状瘢痕期形成
18984                    左眼湿性AMD(部分盘状瘢痕形成);老年性白内障(未熟期)                        左AMD湿性部分盘状瘢痕期形成老年性白内障未熟期
20057   双眼屈光间质混浊

In [52]:
## 黄斑变性AMD——仅含——湿性AMD——补全一级标签
def amd_normal_last_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('AMD').findall(str_i))>0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            str_i_new_list = []
            for k in range(len(str_i_list)):
                if (len(re.compile('湿性AMD|AMD湿性',re.I).findall(str_i_list[k]))>0) and (len(re.compile('黄斑病变',re.I).findall(str_i_list[k]))==0) :  # #替换仅为糖尿病视网膜病变的标签
#                     print(str_i_list)
                    str_i_1 = re.compile('湿性AMD|AMD湿性',re.I).sub('黄斑病变湿性AMD',str_i_list[k])
                elif (len(re.compile('干性AMD|AMD干性',re.I).findall(str_i_list[k]))>0) and (len(re.compile('黄斑病变',re.I).findall(str_i_list[k]))==0):
#                     print(str_i_list)
                    str_i_1 = re.compile('干性AMD|AMD干性',re.I).sub('黄斑病变干性AMD',str_i_list[k])
                elif (len(re.compile('黄斑变性',re.I).findall(str_i_list[k]))==0) and (len(re.compile('黄斑病变',re.I).findall(str_i_list[k]))==0):
#                     print(str_i_list)
                    str_i_1 = re.compile('AMD',re.I).sub('黄斑病变AMD',str_i_list[k])
                else:
                    str_i_1 = str_i_list[k]
                
                str_i_new_list.append(str_i_1)
    #                 print("统一名称后list：{}".format(str_i_new_list))
            df.at[i,'阅_gai'] = '!'.join(str_i_new_list[m] for m in range(len(str_i_new_list)))
#             print('__'+df.at[i,'阅_gai'])
        else:
            pass

In [53]:
amd_normal_last_fuc(df_amd)

In [54]:
df_amd[df_amd['阅_gai'].str.contains('AMD可能大',re.I)]['阅_gai'].value_counts()  

Series([], Name: 阅_gai, dtype: int64)

In [55]:
## 黄斑变性AMD——# 黄斑病变——不同名称顺序的统一化
def amd_normal_te_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('视锥细胞营养不良|AMD干性|AMD湿性|Best|黄斑瘢痕期|黄斑病变瘢痕期|黄斑病变中期|黄斑病变早期|湿性瘢痕期|湿性中期|湿性早期|黄斑病变Stargardt病|AMD瘢痕期|AMD中期|AMD早期|黄斑病变湿性|湿性黄斑病变|黄斑病变干性|干性黄斑病变|黄斑病变AMD|湿性AMD|干性AMD').findall(str_i))>0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#             print(str_i_list)
            str_i_new_list = []
            for k in range(len(str_i_list)):
                str_i_1 = single_replace_fuc('黄斑病变湿性AMD', '黄斑病变_黄斑变性_AMD_湿性', str_i_list[k])
                str_i_2 = single_replace_fuc('湿性黄斑病变AMD', '黄斑病变_黄斑变性_AMD_湿性', str_i_1)
                str_i_3 = single_replace_fuc('黄斑病变干性AMD', '黄斑病变_黄斑变性_AMD_干性', str_i_2)
                str_i_4 = single_replace_fuc('干性黄斑病变AMD', '黄斑病变_黄斑变性_AMD_干性', str_i_3)
                str_i_5 = single_replace_fuc('黄斑病变AMD', '黄斑病变_黄斑变性_AMD', str_i_4)
                str_i_6 = single_replace_fuc('黄斑病变湿性', '黄斑病变_黄斑变性_AMD_湿性', str_i_5)
                str_i_7 = single_replace_fuc('黄斑病变干性', '黄斑病变_黄斑变性_AMD_干性', str_i_6)
                str_i_8 = single_replace_fuc('湿性AMD', 'AMD_湿性', str_i_7)
                str_i_9 = single_replace_fuc('干性AMD', 'AMD_干性', str_i_8)
                str_i_10 = single_replace_fuc('AMD湿性', 'AMD_湿性', str_i_9)
                str_i_11 = single_replace_fuc('AMD干性', 'AMD_干性', str_i_10)              
                str_i_12 = single_replace_fuc('AMD早期', 'AMD_早期', str_i_11)
                str_i_13 = single_replace_fuc('AMD中期', 'AMD_中期', str_i_12)
                str_i_14 = single_replace_fuc('AMD瘢痕期', 'AMD_瘢痕期', str_i_13)
                str_i_15 = single_replace_fuc('黄斑病变Stargardt病', '黄斑病变_黄斑变性_先天性_Stargardt病', str_i_14)  # 黄斑病变Stargardt病
                str_i_16 = single_replace_fuc('湿性早期', '湿性_早期', str_i_15)
                str_i_17 = single_replace_fuc('湿性中期', '湿性_中期', str_i_16)
                str_i_18 = single_replace_fuc('湿性瘢痕期', '湿性_瘢痕期', str_i_17)
                str_i_19 = single_replace_fuc('黄斑病变早期', '黄斑病变_早期', str_i_18)
                str_i_20 = single_replace_fuc('黄斑病变中期', '黄斑病变_中期', str_i_19)
                str_i_21 = single_replace_fuc('黄斑病变瘢痕期', '黄斑病变_瘢痕期', str_i_20)
                str_i_22 = single_replace_fuc('黄斑瘢痕期', '黄斑病变_瘢痕期', str_i_21)
                str_i_23 = single_replace_fuc('Best病', '黄斑病变_黄斑变性_先天性_Best病', str_i_22)
                str_i_24 = single_replace_fuc('视锥细胞营养不良', '黄斑病变_黄斑变性_先天性_视锥细胞营养不良', str_i_23)
#                 str_i_17 = single_replace_fuc('白内障初期', '白内障_初发期', str_i_16)   # 初期——初发期
#                 str_i_18 = single_replace_fuc('白内障中期', '白内障_未熟期', str_i_17)   # 中期——未熟期
#                 str_i_19 = single_replace_fuc('白内障晚期', '白内障_过熟期', str_i_18)   # 晚期——过熟期
#                 str_i_20 = single_replace_fuc('白内障早期', '白内障_初发期', str_i_19)   # 早期——初发期
#                 str_i_21 = single_replace_fuc('早期白内障', '白内障_初发期', str_i_20)
#                 str_i_22 = single_replace_fuc('白内障!左甚', '白内障!左白内障_未熟期', str_i_21)  
                str_i_new_list.append(str_i_24)
#                 print("统一名称后list：{}".format(str_i_new_list))
                df.at[i,'阅_gai'] = '!'.join(str_i_new_list[m] for m in range(len(str_i_new_list)))   # 列表推导式逐个读取字符串列表中的字符串，并将其赋值给df_tw
        else:
            pass
            
    check_df_jb = df[df['阅_gai'].str.contains('AMD干性|AMD湿性|黄斑瘢痕期|黄斑病变瘢痕期|黄斑病变中期|黄斑病变早期|湿性瘢痕期|湿性中期|湿性早期|黄斑病变Stargardt病|AMD瘢痕期|AMD中期|AMD早期|黄斑病变湿性|黄斑病变干性|干性黄斑病变|黄斑病变AMD|湿性AMD|干性AMD')]
    if check_df_jb.shape[0] > 0:
        print("剩余{}条数据未转化为——黄斑病变".format(check_df_jb.shape[0]))
        print(check_df_jb['阅_gai'])
        amd_normal_te_fuc(df)
    else:
        print('转化结束！！！')

In [56]:
amd_normal_te_fuc(df_amd)

转化结束！！！


## 补全一级标签——黄斑病变

In [57]:
# 黄斑变性——补全一级标签——黄斑病变
for i in range(df_amd.shape[0]):
    str_i = str(df_amd.at[i,'阅_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('黄斑变性|黄斑裂孔|黄斑前膜|黄斑水肿|黄斑区色素上皮改变|黄斑区色素上皮脱离|黄斑牵拉综合征|黄斑出血').findall(str_i))>0:
        str_tempt = re.split(r'!', str_i)
        str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
        str_i_new_list = []
        for k in range(len(str_i_list)):
            if len(re.compile('黄斑变性').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑变性', '黄斑病变_黄斑变性', str_i_list[k])
            elif len(re.compile('黄斑裂孔').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑裂孔', '黄斑病变_黄斑裂孔', str_i_list[k])
            elif len(re.compile('黄斑前膜').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑前膜', '黄斑病变_黄斑前膜', str_i_list[k])
            elif len(re.compile('黄斑水肿').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑水肿', '黄斑病变_黄斑水肿', str_i_list[k])
            elif len(re.compile('黄斑区色素上皮改变').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑区色素上皮改变', '黄斑病变_黄斑区色素上皮改变', str_i_list[k])
            elif len(re.compile('黄斑区色素上皮脱离').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑区色素上皮脱离', '黄斑病变_黄斑区色素上皮脱离', str_i_list[k])
            elif len(re.compile('黄斑牵拉综合征').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑牵拉综合征', '黄斑病变_黄斑牵拉综合征', str_i_list[k])
            elif len(re.compile('黄斑出血').findall(str_i_list[k]))>0 and len(re.compile('黄斑病变').findall(str_i_list[k]))==0:
                str_i_1 = single_replace_fuc('黄斑出血', '黄斑病变_黄斑出血', str_i_list[k]) 
            else:
                str_i_1 = str_i_list[k]
                
            str_i_new_list.append(str_i_1)
            
        df_amd.at[i,'阅_gai'] = '!'.join(str_i_new_list[m] for m in range(len(str_i_new_list)))   # 列表推导式逐个读取字符串列表中的字符串，并将其赋值给df_tw            
    else:
        pass

In [58]:
a = df_amd[df_amd['阅_gai'].str.contains('双(.*)黄斑(.*)右')]
a['阅_gai'].value_counts()  # 左黄斑病变_中浆AMD  左黄斑病变PCVAMD   左黄斑病变黄斑病变_黄斑变性_AMD_湿性黄斑病变_中浆  AMD老年性  右外伤!双黄斑变性 

双黄斑病变_黄斑前膜!右黄斑病变_黄斑裂孔                                                                                9
双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_湿性                                                                     7
双黄斑病变_黄斑变性_AMD!右白内障!左白内障摘除人工晶状体植入术后                                                                  5
双黄斑病变_黄斑变性_AMD!右瘢痕期                                                                                  4
双黄斑病变_黄斑变性_AMD!右湿性                                                                                   4
双视网膜动脉硬化!左黄斑病变!右屈光不正                                                                                 4
双黄斑病变_黄斑变性_AMD!右湿性!左干性                                                                               3
双黄斑病变_黄斑变性_AMD_湿性!右瘢痕期形成                                                                             3
双黄斑病变_黄斑前膜!右黄斑病变_黄斑裂孔_板层                                                                             3
双黄斑病变!右黄斑病变_黄斑前膜                                                         

## 左/右几期补全一、二级标签

In [59]:
## 黄斑变性AMD
###### 左干性——结尾的添加父标签时，添加分隔符“！”
#  ["湿性","干性","早期","中期","瘢痕期","外伤"]
def add_last_tan_fuc(df, match_x_lr):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('湿性|干性|早期|中期|瘢痕期|外伤').findall(str_i))>0:
            match_x_list = ["湿性","干性","早期","中期","瘢痕期","外伤"]
            for j in range(len(match_x_list)):
                str_i = str(str_i)    # 在正则表达式之前，转换一下数据类型，出现错误TypeError: expected string or bytes-like object
                if len(re.compile(match_x_lr+match_x_list[j]).findall(str_i))>0:
#                     print(str_i)
                    index_insert = [i.end() for i in re.finditer(match_x_lr+match_x_list[j],str_i)]        
            #         print(index_insert[0])
                    str_i = list(str_i)    # str 无法直接插入，需先转为list
                    str_i.insert(index_insert[0],"!")
                    df.at[i,'阅_gai'] ="".join(str_i)   # 将分隔开的字符合并到一起
#                     print("__{}".format(df.at[i,'阅_gai']))
                else:
                    pass
        else:
            pass

In [60]:
add_last_tan_fuc(df_amd,'左')

In [61]:
add_last_tan_fuc(df_amd,'右')

In [62]:
## 黄斑变性AMD
##### 为“双黄斑病变_黄斑变性_AMD!右湿性!左干性 ”的左右眼添加一级标签  右干性!左史性   
### 双黄斑病变!左外伤  双年龄相关性黄斑变!右早期!左晚期 左活动期!右瘢痕期  
def insert_class_function(a, match_x):
    left, middle, right = a.partition(match_x)   # 使用匹配到的字符进行前中后，3区分隔
    if len(re.compile('黄斑病变',re.I).findall(left)) > 0 and len(re.compile('双',re.I).findall(left)) > 0:
        if len(re.compile('黄斑病变',re.I).findall(right)) == 0 and len(re.compile('湿性|干性|早期|中期|瘢痕期|外伤').findall(right))>0:  
            match_x = right.split('!')[0]   # 截取“左急性闭角型?”字段中“？”之前的部分，同类情形有“！和；”
            print("匹配字符串：{}".format(a))
            if match_x not in ["湿性","干性","早期","中期","瘢痕期","外伤"]:
                pass
            else:
                print("匹配——{}".format(match_x))
                index_match_x = [i.start() for i in re.finditer(match_x,a)]  # 找到匹配字符串的起始位置——索引
#                 print(index_match_x[0])
                a = list(a)
                insert_match_x_strat = [i.start() for i in re.finditer('双黄',left)]
#                 print("__{}".format(insert_match_x_strat))
                insert_match_x_end = [i.start() for i in re.finditer('!',left)]
                insert_match_x_end = [p for p in insert_match_x_end if p > insert_match_x_strat[0]+1 ]  # 列表生成式仅筛选大于insert_match_x_strat[0]索引的“！”
#                 print("**{}".format(insert_match_x_end))
                insert_word_x = left[insert_match_x_strat[0]+1:insert_match_x_end[0]] + '_'

                a.insert(index_match_x[0],insert_word_x)   # 在”左“和“2”的中间插入“一级标签”类别
                a ="".join(a)
                print("插入后：{}".format(a))
        else:
            pass
    
    else:
        pass
        
    return a

In [63]:
## 黄斑变性AMD
# 处理“左几期”的情形：
def lr_jiqi_fuc(df, match_str_x):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('双黄斑病变',re.I).findall(str_i)) == 0:
            pass        
        else:
            df.at[i,'阅_gai'] = insert_class_function(str_i, match_str_x) # 准备匹配检索的字符串——此处为“左2期”或“右1期”等
#             print("__{}".format(df.at[i,'阅_gai']))

#     check_df_jb_0 = df[df['阅_gai'].str.contains('双青光眼')] 
#     check_df_jb = check_df_jb_0[check_df_jb_0['阅_gai'].str.contains(match_str_x+'继发|开角|闭角|新生血管|绝对期')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] > 0:
#         print("剩余{}条数据未转化为——青光眼".format(check_df_jb.shape[0]))
#         print(check_df_jb[['阅片诊断','阅_gai']])
# #         insert_class_function(df, match_x)
#     else:
#         print('转化结束！！！')         

In [64]:
lr_jiqi_fuc(df_amd, '左')   

匹配字符串：双黄斑病变_黄斑变性_AMD!左湿性!屈光不正
匹配——湿性
插入后：双黄斑病变_黄斑变性_AMD!左黄斑病变_黄斑变性_AMD_湿性!屈光不正
匹配字符串：双黄斑病变_黄斑变性_AMD!右湿性!!左干性!可疑青光眼
匹配——干性
插入后：双黄斑病变_黄斑变性_AMD!右湿性!!左黄斑病变_黄斑变性_AMD_干性!可疑青光眼
匹配字符串：双屈光间质混浊白内障可能性大!双黄斑病变黄斑病变_黄斑变性_AMD!右干性!!左瘢痕期!改变
匹配——瘢痕期
插入后：双屈光间质混浊白内障可能性大!双黄斑病变黄斑病变_黄斑变性_AMD!右干性!!左黄斑病变黄斑病变_黄斑变性_AMD_瘢痕期!改变
匹配字符串：双高度近视!双黄斑病变!右黄斑病变_黄斑变性_AMD_湿性!左瘢痕期!
匹配——瘢痕期
插入后：双高度近视!双黄斑病变!右黄斑病变_黄斑变性_AMD_湿性!左黄斑病变_瘢痕期!
匹配字符串：双黄斑病变_黄斑变性_AMD!右湿性!!左干性!!双白内障
匹配——干性
插入后：双黄斑病变_黄斑变性_AMD!右湿性!!左黄斑病变_黄斑变性_AMD_干性!!双白内障
匹配字符串：双老年性白内障皮质性!双黄斑病变_黄斑变性_AMD!右早期!!左湿性!
匹配——湿性
插入后：双老年性白内障皮质性!双黄斑病变_黄斑变性_AMD!右早期!!左黄斑病变_黄斑变性_AMD_湿性!
匹配字符串：双黄斑病变_黄斑变性_AMD!右早期!!左湿性!
匹配——湿性
插入后：双黄斑病变_黄斑变性_AMD!右早期!!左黄斑病变_黄斑变性_AMD_湿性!
匹配字符串：双黄斑病变_黄斑变性_AMD_湿性!左瘢痕期!!双IOL眼
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD_湿性!左黄斑病变_黄斑变性_AMD_湿性_瘢痕期!!双IOL眼
匹配字符串：双老年性白内障!双青光眼待排!双黄斑病变_黄斑变性_AMD!右湿性!!左干性!!双屈光不正
匹配——干性
插入后：双老年性白内障!双青光眼待排!双黄斑病变_黄斑变性_AMD!右湿性!!左黄斑病变_黄斑变性_AMD_干性!!双屈光不正
匹配字符串：双黄斑病变_黄斑变性_AMD!右干性!!左瘢痕期!
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右干性!!左黄斑病变_黄斑变性_AMD_瘢痕期!
匹配字符串：双人工晶体!双黄斑病变_黄斑变性_AMD可能

In [65]:
lr_jiqi_fuc(df_amd, '右')     

匹配字符串：双黄斑病变_黄斑变性_AMD_湿性!右瘢痕期!形成
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD_湿性!右黄斑病变_黄斑变性_AMD_湿性_瘢痕期!形成
匹配字符串：双黄斑病变_黄斑变性_AMD_干性!右瘢痕期!!双老年性白内障
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD_干性!右黄斑病变_黄斑变性_AMD_干性_瘢痕期!!双老年性白内障
匹配字符串：双黄斑病变_黄斑变性_AMD!右瘢痕期!形成
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!形成
匹配字符串：双老年性白内障!双黄斑病变_黄斑变性_AMD_干性!右瘢痕期!!双屈光不正
匹配——瘢痕期
插入后：双老年性白内障!双黄斑病变_黄斑变性_AMD_干性!右黄斑病变_黄斑变性_AMD_干性_瘢痕期!!双屈光不正
匹配字符串：双黄斑病变_黄斑变性_AMD_湿性!右瘢痕期!形成
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD_湿性!右黄斑病变_黄斑变性_AMD_湿性_瘢痕期!形成
匹配字符串：双黄斑病变_黄斑变性_AMD!右瘢痕期!
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!
匹配字符串：双黄斑病变_黄斑变性_AMD!右瘢痕期!!左活动期
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!!左活动期
匹配字符串：双黄斑病变_黄斑变性_AMD!右瘢痕期!化
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!化
匹配字符串：双黄斑病变_黄斑变性_AMD_湿性PCV不除外!右瘢痕期!形成!右IOL眼!左白内障
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD_湿性PCV不除外!右黄斑病变_黄斑变性_AMD_湿性PCV不除外_瘢痕期!形成!右IOL眼!左白内障
匹配字符串：双黄斑病变_黄斑变性_AMD!右瘢痕期!
匹配——瘢痕期
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!
匹配字符串：双黄斑病变_黄斑变性_AMD!右干性!!左青光眼!双视网膜动脉硬化
匹配——干性
插入后：双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_干性!!左青光眼!双视网膜动脉硬化
匹配字符

In [66]:
a = df_amd[df_amd['阅_gai'].str.contains('瘢痕期')]
a['阅_gai'].value_counts()  # 左黄斑病变_中浆AMD  左黄斑病变PCVAMD  

左黄斑病变_瘢痕期                                                              4
双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!                                     4
双黄斑病变_黄斑变性_AMD_湿性_瘢痕期                                                  4
右黄斑病变_黄斑变性_AMD_湿性_瘢痕期形成                                                3
双黄斑病变_黄斑变性_AMD_湿性!右黄斑病变_黄斑变性_AMD_湿性_瘢痕期!形成                             3
右黄斑病变_瘢痕期                                                              3
双黄斑病变_黄斑变性_AMD_瘢痕期                                                     2
左黄斑病变_黄斑变性_AMD_湿性_瘢痕期                                                  2
双黄斑病变_黄斑变性_AMD_湿性!左活动期!右黄斑病变_黄斑变性_AMD_湿性_瘢痕期!                          2
左黄斑病变_黄斑变性_AMD_湿性_瘢痕期形成                                                2
左黄斑病变_黄斑变性_AMD_瘢痕期                                                     2
双黄斑病变_黄斑变性_AMD_湿性_瘢痕期形成                                                2
双高度近视眼底改变!右黄斑病变_瘢痕期!左点状内层脉络膜病变                                         2
双老年性白内障!右黄斑病变_黄斑变性_AMD_瘢痕期+复发!双底玻璃膜疣!双屈光不正         

In [67]:
df_amd[df_amd['阅_gai'].str.contains('囊样变性')]['阅_gai'].value_counts()

左视网膜分支静脉阻塞!双视网膜动脉硬化!左黄斑病变_黄斑囊样变性                          1
左视网膜分支静脉阻塞!左底光凝术后!左黄斑病变_黄斑囊样变性!双视网膜动脉硬化                   1
双视网膜动脉硬化!双视神经萎缩!双青光眼待排!右底光凝术后!右黄斑病变_黄斑囊样变性                1
双视网膜动脉硬化!右视网膜分支静脉阻塞陈旧!右黄斑病变_黄斑囊样变性!右底光凝术后!双青光眼待排!双屈光不正    1
双青光眼待排!右黄斑病变_黄斑囊样变性!右黄斑病变_黄斑出血!双屈光不正                      1
双视神经萎缩脑瘤术后!左黄斑病变_黄斑囊样变性                                   1
右白内障术后眼底出血黄斑病变_黄斑囊样变性!左老年性白内障                             1
Name: 阅_gai, dtype: int64

## 其余特殊情形标签的归一化

In [68]:
## 黄斑变性AMD
## 黄斑病变——不同名称顺序的统一化
def bnz_shunxu_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('黄斑裂孔假性|黄斑病变|湿性晚期|黄斑前膜\+黄斑病变_黄斑裂孔|黄斑水肿\+|_湿性黄斑病变_').findall(str_i))>0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#             print(str_i_list)
            str_i_new_list = []
            for k in range(len(str_i_list)):
                str_i_1 = single_replace_fuc('黄斑病变_黄斑变性_AMD_湿性待查黄斑病变_中浆', '黄斑病变_黄斑变性_AMD_湿性+黄斑病变_中浆', str_i_list[k])
                str_i_2 = single_replace_fuc('湿性晚期', '湿性_瘢痕期', str_i_1)
                str_i_3 = single_replace_fuc('黄斑前膜+黄斑病变_黄斑裂孔', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔', str_i_2)
                str_i_4 = single_replace_fuc('黄斑水肿+黄斑病变_黄斑前膜', '黄斑病变_黄斑水肿+黄斑病变_黄斑前膜', str_i_3)
                str_i_5 = single_replace_fuc('黄斑前膜+黄斑病变_黄斑裂孔_假性', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔_假性', str_i_4)
                str_i_6 = single_replace_fuc('黄斑前膜+黄斑病变_黄斑裂孔_假性', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔_假性', str_i_5)
                str_i_7 = single_replace_fuc('黄斑前膜+黄斑病变_黄斑裂孔_板层', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔_板层', str_i_6)
                str_i_8 = single_replace_fuc('黄斑病变黄斑前膜+黄斑裂孔', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔', str_i_7)
                str_i_9 = single_replace_fuc('黄斑病变PCVAMD', '黄斑病变_黄斑变性_AMD', str_i_8)
                str_i_10 = single_replace_fuc('黄斑病变待定AMD', '黄斑病变_黄斑变性_AMD', str_i_9)
                str_i_11 = single_replace_fuc('黄斑病变黄斑牵拉综合征', '黄斑病变_黄斑牵拉综合征', str_i_10)  # 先天性黄斑病变黄斑缺损
                str_i_12 = single_replace_fuc('黄斑病变WAMD', '黄斑病变_黄斑变性_AMD', str_i_11)  # 先天性黄斑病变黄斑缺损
                str_i_13 = single_replace_fuc('黄斑牵拉综合征+黄斑病变_黄斑水肿', '黄斑病变_黄斑牵拉综合征+黄斑病变_黄斑水肿', str_i_12)
                str_i_14 = single_replace_fuc('黄斑前膜+黄斑病变_黄斑裂孔_假性', '黄斑病变_黄斑前膜+黄斑病变_黄斑裂孔_假性', str_i_13)
                str_i_15 = single_replace_fuc('黄斑病变_黄斑变性_AMD_AMD_湿性', '黄斑病变_黄斑变性_AMD_湿性', str_i_14)
                str_i_16 = single_replace_fuc('_湿性黄斑病变_', '_湿性+黄斑病变_', str_i_15)
                str_i_17 = single_replace_fuc('遗传性黄斑病变_营养不良', '黄斑病变_营养不良_先天性', str_i_16)
                str_i_18 = single_replace_fuc('黄斑病变_中浆AMD', '黄斑病变_中浆+黄斑病变_黄斑变性_AMD', str_i_17)
                str_i_18 = single_replace_fuc('黄斑病变_中浆AMD', '黄斑病变_中浆+黄斑病变_黄斑变性_AMD', str_i_17)
                str_i_19 = single_replace_fuc('黄斑裂孔假性', '黄斑裂孔_假性', str_i_18)
                
                str_i_new_list.append(str_i_19)
#                 print("统一名称后list：{}".format(str_i_new_list))
            df.at[i,'阅_gai'] = '!'.join(str_i_new_list[m] for m in range(len(str_i_new_list)))   # 列表推导式逐个读取字符串列表中的字符串，并将其赋值给df_tw
#             print("统一名称后list：{}".format(df.at[i,'阅_gai']))
        else:
            pass
            
    check_df_jb = df[df['阅_gai'].str.contains('黄斑裂孔假性|黄斑病变_黄斑变性_AMD_湿性待查黄斑病变_中浆')]
    if check_df_jb.shape[0] > 0:
        print("剩余{}条数据未转化为——复合型——黄斑病变".format(check_df_jb.shape[0]))
        print(check_df_jb[['阅片诊断','阅_gai']])
#         bnz_shunxu_normal_fuc(df)
    else:
        print('转化结束！！！')

In [69]:
bnz_shunxu_normal_fuc(df_amd)

转化结束！！！


In [70]:
# df_amd[df_amd['阅_gai'].str.contains('黄斑前膜+黄斑病变')]

## 校验标签分级修改完全

In [71]:
for i in range(df_amd.shape[0]):
    str_i = str(df_amd.at[i,'阅_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('AMD干性|AMD湿性|黄斑瘢痕期|黄斑病变瘢痕期|黄斑病变中期|黄斑病变早期|湿性瘢痕期|湿性中期|湿性早期|黄斑病变Stargardt病|AMD瘢痕期|AMD中期|AMD早期|黄斑病变湿性|湿性黄斑病变|黄斑病变干性|干性黄斑病变|黄斑病变AMD|湿性AMD|干性AMD',re.I).findall(str_i)) > 0:
        print(str_i)
#         bnz_normal_te_fuc(df_amd)
#         bnz_normal_fuc(df_amd)
#         lr_jiqi_fuc(df_amd, '左')  
#         lr_jiqi_fuc(df_amd, '右')
#         bnz_shunxu_normal_fuc(df_amd)
    else:
        pass

In [72]:
df_amd[df_amd['阅_gai'].str.contains('中浆(.*)中渗',re.I)]['阅_gai'].value_counts()  

左陈旧复发性黄斑病变_中浆!左黄斑病变_中渗                             1
右黄斑病变_中浆/黄斑病变_中渗                                   1
左黄斑病变_中浆不除外中渗病                                     1
左黄斑病变_中浆复发黄斑病变_中渗待除外                               1
右黄斑病变_中浆可能黄斑病变_中渗不除外                               1
左黄斑病变以黄斑病变_中浆病或特发性黄斑病变_黄斑变性_AMD_湿性+黄斑病变_中渗病可能性大    1
Name: 阅_gai, dtype: int64

## 添加新标签

In [73]:
df_amd.insert(df_amd.columns.get_loc('阅_gai')+1,'right_黄斑病变',None)
df_amd.insert(df_amd.columns.get_loc('阅_gai')+1,'left_黄斑病变',None)

In [74]:
# 定义——主体化标签——统一的糖网可能情形列表
column_tw = ['黄斑病变','黄斑病变_中浆','黄斑病变_中渗','黄斑病变_黄斑变性','黄斑病变_高度近视','黄斑病变_外伤性','黄斑病变_黄斑前膜',
             '黄斑病变_黄斑裂孔','黄斑病变_黄斑出血','黄斑病变_黄斑水肿','黄斑病变_营养不良','黄斑病变_黄斑牵拉综合征',
             '黄斑病变_黄斑区色素上皮改变','黄斑病变_黄斑区色素上皮脱离','黄斑病变_黄斑囊样变性',
             '黄斑病变_瘢痕期','黄斑病变_早期','黄斑病变_中期',
             '黄斑病变_黄斑裂孔_继发性','黄斑病变_黄斑前膜_继发性','黄斑病变_黄斑前膜_局灶性','黄斑病变_黄斑裂孔_假性',
             '黄斑病变_黄斑前膜_早期','黄斑病变_黄斑前膜_中期','黄斑病变_黄斑前膜_晚期',
             '黄斑病变_黄斑水肿_黄斑囊样水肿','黄斑病变_黄斑水肿_糖尿病性','黄斑病变_黄斑裂孔_板层','黄斑病变_黄斑裂孔_全层',
             '黄斑病变_黄斑变性_AMD','黄斑病变_营养不良_先天性','黄斑病变_营养不良_萎缩性',
             '黄斑病变_黄斑变性_AMD_湿性','黄斑病变_黄斑变性_AMD_干性',
             '黄斑病变_黄斑变性_AMD_早期','黄斑病变_黄斑变性_AMD_中期','黄斑病变_黄斑变性_AMD_瘢痕期',
             '黄斑病变_黄斑水肿_糖尿病性_轻度','黄斑病变_黄斑水肿_糖尿病性_中度','黄斑病变_黄斑水肿_糖尿病性_重度',
             '黄斑病变_黄斑裂孔_特发性_全层','黄斑病变_黄斑裂孔_特发性_板层',
             '黄斑病变_黄斑变性_先天性_Stargardt病','黄斑病变_黄斑变性_先天性_Best病','黄斑病变_黄斑变性_先天性_视锥细胞营养不良',
             '黄斑病变_黄斑变性_AMD_湿性_早期','黄斑病变_黄斑变性_AMD_湿性_中期','黄斑病变_黄斑变性_AMD_湿性_瘢痕期']

In [75]:
# 添加黄斑病变的解析标签
def add_new_label_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'阅_gai'])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            for j in range(len(column_tw)):
                column_tw_j = column_tw[j]
                column_tw_j_list = column_tw_j.split('_')
    #             print(column_tw_j_list)       
                for k in range(len(str_i_list)):
                    if (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'双')!=-1):
        #                     print(str_i_list[k]) .join(str_i_new_list[m] for m in range(len(str_i_new_list)))
                        df.at[i,'left_黄斑病变'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                        df.at[i,'right_黄斑病变'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    elif (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'左')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'left_黄斑病变'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    elif (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'右')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'right_黄斑病变'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    else:
                        pass

In [76]:
add_new_label_fuc(df_amd)   # 调用添加新标签的函数

### 标签修正，单眼罹患多种黄斑病变的疾病种类~

In [77]:
# 定义——副的主体化标签——排除“黄斑病变”，避免删除函数涉及“黄斑病变”的删除
column_tw_fu = ['黄斑病变_中浆','黄斑病变_中渗','黄斑病变_黄斑变性','黄斑病变_高度近视','黄斑病变_外伤性','黄斑病变_黄斑前膜',
             '黄斑病变_黄斑裂孔','黄斑病变_黄斑出血','黄斑病变_黄斑水肿','黄斑病变_营养不良','黄斑病变_黄斑牵拉综合征',
             '黄斑病变_黄斑区色素上皮改变','黄斑病变_黄斑区色素上皮脱离','黄斑病变_黄斑囊样变性',
             '黄斑病变_瘢痕期','黄斑病变_早期','黄斑病变_中期',
             '黄斑病变_黄斑裂孔_继发性','黄斑病变_黄斑前膜_继发性','黄斑病变_黄斑前膜_局灶性','黄斑病变_黄斑裂孔_假性',
             '黄斑病变_黄斑前膜_早期','黄斑病变_黄斑前膜_中期','黄斑病变_黄斑前膜_晚期',
             '黄斑病变_黄斑水肿_黄斑囊样水肿','黄斑病变_黄斑水肿_糖尿病性','黄斑病变_黄斑裂孔_板层','黄斑病变_黄斑裂孔_全层',
             '黄斑病变_黄斑变性_AMD','黄斑病变_营养不良_先天性','黄斑病变_营养不良_萎缩性',
             '黄斑病变_黄斑变性_AMD_湿性','黄斑病变_黄斑变性_AMD_干性',
             '黄斑病变_黄斑变性_AMD_早期','黄斑病变_黄斑变性_AMD_中期','黄斑病变_黄斑变性_AMD_瘢痕期',
             '黄斑病变_黄斑水肿_糖尿病性_轻度','黄斑病变_黄斑水肿_糖尿病性_中度','黄斑病变_黄斑水肿_糖尿病性_重度',
             '黄斑病变_黄斑裂孔_特发性_全层','黄斑病变_黄斑裂孔_特发性_板层',
             '黄斑病变_黄斑变性_先天性_Stargardt病','黄斑病变_黄斑变性_先天性_Best病','黄斑病变_黄斑变性_先天性_视锥细胞营养不良',
             '黄斑病变_黄斑变性_AMD_湿性_早期','黄斑病变_黄斑变性_AMD_湿性_中期','黄斑病变_黄斑变性_AMD_湿性_瘢痕期']

In [78]:
# 删除重复字符串函数
def delect_fuc(x):
    str_x = ['黄斑变性','AMD','湿性','干性','先天性','黄斑裂孔','特发性','黄斑前膜','黄斑水肿','糖尿病性','营养不良'] # 二级标签列表
    y = {}  # 设置查重字符串字典
    check_count = {}
    for i in range(len(str_x)):
        y[str_x[i]] = x.count(str_x[i])
#         print("重复字符串：{}".format(y))
        if y[str_x[i]] > 1:
            duplicate_start_x = [j.start() for j in re.finditer(str_x[i],x)]
#             print(duplicate_start_x)
            separate_list = [j.start() for j in re.finditer(';',x)]
#             print(separate_list)
            if separate_list[0] < duplicate_start_x[0]:  # 非重复标签开头
                index_start = [j for j in separate_list if j < duplicate_start_x[0]][-1]  # 起始索引——截取小于min重复词的；中，索引值最大的一个。
                print("删除字符串的起始索引：{}".format(index_start))
                index_end = [k for k in separate_list if k < duplicate_start_x[-1]][-1]   # 结束索引——截取小于max重复词；中，索引值最大的一个。
                print("删除字符串的结束索引：{}".format(index_end))   
                x = x[:index_start] + x[index_end:]
                print("删除多余字符之后的标签项——x：{}".format(x))
            else:  # 重复标签开头
                index_start = 0
                print("删除字符串的起始索引：{}".format(index_start))
                index_end = [k for k in separate_list if k < duplicate_start_x[-1]][-1]+1   # 结束索引——截取小于max重复词；中，索引值最大的一个。
                print("删除字符串的结束索引：{}".format(index_end))   
                x = x[:index_start] + x[index_end:]
                print("删除多余字符之后的标签项——x：{}".format(x))
        else:
            pass
    return x 

In [79]:
# y = '黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,湿性'
# y_split_list = y.split(';')
# print(y_split_list)
# print([len(y_split_list[i]) for i in range(len(y_split_list))])
# z = delect_fuc(y)
# z

In [91]:
# "必须含双，可能含左、右含多种疾病的打标"函数
def str_fix_fuc(a):
    str_tempt = re.split(r'!',a)
    str_i_list = list(filter(None,str_tempt))
    print(str_i_list)
    for j in range(len(str_i_list)):
        print(str_i_list[j])
        for i in range(len(column_tw_fu)):
            column_tw_fu_i_list = column_tw_fu[i].split('_')
            if len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('双').findall(str_i_list[j])) > 0:
                r = l = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                print("d============={}".format(r))
            elif len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('右').findall(str_i_list[j])) > 0:
                d1 = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                r = ';'.join([r,d1])
                print("r**************{}".format(r))
            elif len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('左').findall(str_i_list[j])) > 0:
                d2 = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
#                 print(d2)
                l = ';'.join([l,d2])
                print("l~~~~~~~~~~~~~~~{}".format(l))
            else:
                pass
    r = delect_fuc(r)
    l = delect_fuc(l)
                
    return r,l

In [81]:
# df_amd[df_amd['阅_gai'].str.contains('左黄斑病变_(.*)左黄斑(.*)',re.I)]['阅_gai'].value_counts()  

In [92]:
# 处理“左、左”或“右、右”多种疾病的情形函数
def str_fix_single_fuc(a,mtch_qz):  # mtch_qz为匹配的前缀，a为匹配的str_n
    str_tempt = re.split(r'!',a)
    str_i_list = list(filter(None,str_tempt))
    print(str_i_list)
    l_list = []
    for j in range(len(str_i_list)):
        print(str_i_list[j])
        l_list_x = []   # 存放单一的“左黄斑病变_黄斑变性_AMD_湿性可能性大”项
        for i in range(len(column_tw_fu)):
            column_tw_fu_i_list = column_tw_fu[i].split('_')
            if len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile(mtch_qz).findall(str_i_list[j])) > 0:
                d = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                l_list_x.append(d)
            else:
                pass
#         print("***********{}".format(l_list))
        l1 = ';'.join(i for i in l_list_x)
        l1 = delect_fuc(l1)
#         print("l1{}".format(l1))
        l_list.append(l1)
#         print("=============l{}".format(l_list))
    l = ';'.join( k for k in l_list )               
    return l

In [83]:
# y = '左陈旧复发性黄斑病变_中浆!左黄斑病变_中渗'
# # y = '左黄斑病变_黄斑前膜!左黄斑病变_黄斑裂孔_假性待查!双玻璃体浑浊待查'
# z = str_fix_single_fuc(y,'左')
# z

In [84]:
# for n in range(df_amd.shape[0]):
#     str_n = str(df_amd.at[n,'阅_gai'])
#     if str_n=='nan':   # 排除str_i为”null“的项
#         pass
#     elif len(re.compile('左(.*)黄斑病变_(.*)左(.*)黄斑(.*)').findall(str_n))>0:
#         df_amd.at[n,'left_黄斑病变'] = str_fix_single_fuc(str_n,'左')
#         print("******{}".format(df_amd.at[i,'left_黄斑病变'] ))

In [85]:
# df_amd['left_黄斑病变'].value_counts()

In [86]:
# y_temp = y.split('!')
# print(y_temp)
# index_list = [ i for i in range(len(y_temp)) if re.compile('黄斑病变').findall(y_temp[i]) ]
# print(index_list)
# tempt_a = len(y_temp[index_list[0]])
# tempt_b = len(y_temp[index_list[1]])
# print(tempt_a,tempt_b)
# if tempt_a < tempt_b:  # 寻找字符串短的为子集，长的为全集
#     min_index = y_temp[index_list[0]] 
#     max_index = y_temp[index_list[1]]
# else:
#     min_index = y_temp[index_list[1]]
#     max_index = y_temp[index_list[0]]
# print(min_index,max_index)

# if re.compile(min_index).findall(max_index):
#     print('包含')

In [93]:
# 修正每一眼罹患多种疾病的标签
def fix_new_label_fuc(df):
    for n in range(df.shape[0]):
        str_n = str(df.at[n,'阅_gai'])
        if str_n=='nan':   # 排除str_i为”null“的项
            pass
        elif len(re.compile('双黄斑病变_(.*)左黄斑(.*)|双黄斑病变_(.*)右黄斑(.*)',re.I).findall(str_n))>0:
            print(str_n)
            df.at[n,'right_黄斑病变'],df.at[n,'left_黄斑病变'] = str_fix_fuc(str_n)
            print(df.at[n,'right_黄斑病变']+"__"+df.at[n,'left_黄斑病变'])
        elif len(re.compile('左(.*)黄斑病变_(.*)左(.*)黄斑(.*)').findall(str_n))>0:
            y_temp = str_n.split('!')
            index_list = [ i for i in range(len(y_temp)) if re.compile('黄斑病变').findall(y_temp[i]) ]
            print(index_list)
            if len(index_list) < 2:
                pass
            else:
                tempt_a = len(y_temp[index_list[0]])
                tempt_b = len(y_temp[index_list[1]])
#                 print(tempt_a,tempt_b)
                if tempt_a < tempt_b:  # 寻找字符串短的为子集，长的为全集
                    min_index = y_temp[index_list[0]] 
                    max_index = y_temp[index_list[1]]
                else:
                    min_index = y_temp[index_list[1]]
                    max_index = y_temp[index_list[0]]
                print(min_index,max_index)
                if re.compile(min_index).findall(max_index):  # 包含子集，即左——左——疾病为包含关系
                    pass
                else:
                    df.at[n,'left_黄斑病变'] = str_fix_single_fuc(str_n,'左')
        elif len(re.compile('右(.*)黄斑病变_(.*)右(.*)黄斑(.*)').findall(str_n))>0:
            y_temp = str_n.split('!')
            index_list = [ i for i in range(len(y_temp)) if re.compile('黄斑病变').findall(y_temp[i]) ]
            print(index_list)
            if len(index_list) < 2:
                pass
            else:
                tempt_a = len(y_temp[index_list[0]])
                tempt_b = len(y_temp[index_list[1]])
    #             print(tempt_a,tempt_b)
                if tempt_a < tempt_b:  # 寻找字符串短的为子集，长的为全集
                    min_index = y_temp[index_list[0]] 
                    max_index = y_temp[index_list[1]]
                else:
                    min_index = y_temp[index_list[1]]
                    max_index = y_temp[index_list[0]]
                print("min_index:{},max_index:{}".format(min_index,max_index))
                if re.compile(min_index).findall(max_index):  # 包含子集，即左——左——疾病为包含关系
                    pass
                else:
                    df.at[n,'right_黄斑病变'] = str_fix_single_fuc(str_n,'右')
        else:
            pass

In [94]:
fix_new_label_fuc(df_amd)

[1, 2]
min_index:右黄斑病变_黄斑前膜,max_index:右黄斑病变_黄斑裂孔
['双屈光间质不清', '右黄斑病变_黄斑裂孔', '右黄斑病变_黄斑前膜']
双屈光间质不清
右黄斑病变_黄斑裂孔
右黄斑病变_黄斑前膜
双黄斑病变_黄斑水肿!右黄斑病变_黄斑前膜!右网膜出血
['双黄斑病变_黄斑水肿', '右黄斑病变_黄斑前膜', '右网膜出血']
双黄斑病变_黄斑水肿
d=============黄斑病变,黄斑水肿
右黄斑病变_黄斑前膜
r**************黄斑病变,黄斑水肿;黄斑病变,黄斑前膜
右网膜出血
黄斑病变,黄斑水肿;黄斑病变,黄斑前膜__黄斑病变,黄斑水肿
[0, 1]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑裂孔
['左黄斑病变_黄斑裂孔', '左黄斑病变_黄斑前膜', '双高血压性视网膜病变']
左黄斑病变_黄斑裂孔
左黄斑病变_黄斑前膜
双高血压性视网膜病变
双黄斑病变_黄斑变性_AMD_湿性!右黄斑病变_黄斑变性_AMD_湿性_瘢痕期!形成
['双黄斑病变_黄斑变性_AMD_湿性', '右黄斑病变_黄斑变性_AMD_湿性_瘢痕期', '形成']
双黄斑病变_黄斑变性_AMD_湿性
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
d=============黄斑病变,黄斑变性,AMD,湿性
右黄斑病变_黄斑变性_AMD_湿性_瘢痕期
r**************黄斑病变,黄斑变性,AMD,湿性;黄斑病变,黄斑变性
r**************黄斑病变,黄斑变性,AMD,湿性;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
r**************黄斑病变,黄斑变性,AMD,湿性;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,湿性
r**************黄斑病变,黄斑变性,AMD,湿性;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,湿性;黄斑病变,黄斑变性,AMD,湿性,瘢痕期
形成
删除字符串的起始索引：0
删除字符串的结束索引：58
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性,瘢痕期
黄斑病变,黄斑变性,AMD,湿性,瘢痕期__黄斑病变,黄斑变性,AMD,湿性
双黄

形成
删除字符串的起始索引：0
删除字符串的结束索引：58
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性,瘢痕期
黄斑病变,黄斑变性,AMD,湿性,瘢痕期__黄斑病变,黄斑变性,AMD,湿性
[0, 1]
min_index:右黄斑病变_黄斑裂孔,max_index:右黄斑病变_黄斑前膜
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑裂孔
[0, 1]
min_index:右黄斑病变_黄斑裂孔,max_index:右黄斑病变_黄斑前膜
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔', '左白内障']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑裂孔
左白内障
双视网膜色素变性!双黄斑病变_黄斑前膜!左黄斑病变_黄斑裂孔
['双视网膜色素变性', '双黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔']
双视网膜色素变性
双黄斑病变_黄斑前膜
d=============黄斑病变,黄斑前膜
左黄斑病变_黄斑裂孔
l~~~~~~~~~~~~~~~黄斑病变,黄斑前膜;黄斑病变,黄斑裂孔
黄斑病变,黄斑前膜__黄斑病变,黄斑前膜;黄斑病变,黄斑裂孔
双黄斑病变_黄斑变性_AMD_干性!右黄斑地图样萎缩
['双黄斑病变_黄斑变性_AMD_干性', '右黄斑地图样萎缩']
双黄斑病变_黄斑变性_AMD_干性
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
d=============黄斑病变,黄斑变性,AMD,干性
右黄斑地图样萎缩
黄斑病变,黄斑变性,AMD,干性__黄斑病变,黄斑变性,AMD,干性
[1, 2]
左黄斑病变 右黄斑病变_黄斑前膜
['左青光眼待排', '右黄斑病变_黄斑前膜', '左黄斑病变', '双屈光不正']
左青光眼待排
右黄斑病变_黄斑前膜
左黄斑病变
双屈光不正
双黄斑病变_黄斑变性_AMD!右早期!左黄斑病变_黄斑变性_AMD_湿性
['双黄斑病变_黄斑变性_AMD', '右早期', '左黄斑病变_黄斑变性_AMD_湿性']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右早期
左黄斑病变_黄斑变性_AMD_湿性
l~~~~~~~~~~

[0, 1, 2]
右黄斑病变_瘢痕期 左黄斑病变_黄斑裂孔
['右黄斑病变_瘢痕期', '左黄斑病变_黄斑裂孔', '左黄斑病变_黄斑前膜', '双高血压视网膜病变II']
右黄斑病变_瘢痕期
左黄斑病变_黄斑裂孔
左黄斑病变_黄斑前膜
双高血压视网膜病变II
双人工晶体!双黄斑病变_黄斑变性_AMD可能性大!右湿性!左黄斑病变_黄斑变性_AMD可能性大_干性
['双人工晶体', '双黄斑病变_黄斑变性_AMD可能性大', '右湿性', '左黄斑病变_黄斑变性_AMD可能性大_干性']
双人工晶体
双黄斑病变_黄斑变性_AMD可能性大
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右湿性
左黄斑病变_黄斑变性_AMD可能性大_干性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
删除字符串的起始索引：0
删除字符串的结束索引：24
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD
黄斑病变,黄斑变性,AMD__黄斑病变,黄斑变性,AMD
双黄斑病变_黄斑变性_AMD!左黄斑病变_黄斑变性_AMD_湿性
['双黄斑病变_黄斑变性_AMD', '左黄斑病变_黄斑变性_AMD_湿性']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
左黄斑病变_黄斑变性_AMD_湿性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,湿性
删除字符串的起始索引：0
删除字符串的结束索引：38
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性
黄斑病变,黄斑变性,AMD__黄斑病变,黄斑变性,AMD,湿性
[0, 1, 2]
左黄斑病变_黄斑变性_AMD_干性 右黄斑病变_黄斑变性_AMD_湿性
['右黄斑病变_黄

双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_湿性
['双黄斑病变_黄斑变性_AMD', '右黄斑病变_黄斑变性_AMD_湿性']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右黄斑病变_黄斑变性_AMD_湿性
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,湿性
删除字符串的起始索引：0
删除字符串的结束索引：38
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性
黄斑病变,黄斑变性,AMD,湿性__黄斑病变,黄斑变性,AMD
[0, 1]
min_index:右黄斑病变_黄斑前膜,max_index:右黄斑病变_黄斑变性_AMD_干性
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑变性_AMD_干性', '双病理性近视']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑变性_AMD_干性
删除字符串的起始索引：0
删除字符串的结束索引：24
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,干性
双病理性近视
双黄斑病变_黄斑变性!左黄斑病变_黄斑前膜!双屈光间质浑浊
['双黄斑病变_黄斑变性', '左黄斑病变_黄斑前膜', '双屈光间质浑浊']
双黄斑病变_黄斑变性
d=============黄斑病变,黄斑变性
左黄斑病变_黄斑前膜
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性;黄斑病变,黄斑前膜
双屈光间质浑浊
黄斑病变,黄斑变性__黄斑病变,黄斑变性;黄斑病变,黄斑前膜
[0, 1]
min_index:右黄斑病变_黄斑前膜,max_index:右黄斑病变_黄斑牵拉综合征
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑牵拉综合征', '双高血压性视网膜病变']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑牵拉综合征
双高血压性视网膜病变
双黄斑病变_黄斑变性_AMD!左黄斑病变_黄斑变性_AMD_瘢痕期
['双黄斑病变_黄斑变性_AMD', '左黄斑病变_黄斑变性_

[0, 1]
min_index:右黄斑病变_黄斑前膜,max_index:右可疑黄斑病变_黄斑变性_AMD
['右可疑黄斑病变_黄斑变性_AMD', '右黄斑病变_黄斑前膜', '双视网膜动脉硬化']
右可疑黄斑病变_黄斑变性_AMD
删除字符串的起始索引：0
删除字符串的结束索引：10
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD
右黄斑病变_黄斑前膜
双视网膜动脉硬化
[0, 1]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑变性_AMD_湿性可能性大
['左黄斑病变_黄斑变性_AMD_湿性可能性大', '左黄斑病变_黄斑前膜', '双高血压视网膜病变', '双白内障']
左黄斑病变_黄斑变性_AMD_湿性可能性大
删除字符串的起始索引：0
删除字符串的结束索引：24
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性
左黄斑病变_黄斑前膜
双高血压视网膜病变
双白内障
[1, 2]
min_index:右黄斑病变_黄斑裂孔,max_index:右黄斑病变_黄斑前膜
['双玻璃体后脱离', '右黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔', '双屈光不正']
双玻璃体后脱离
右黄斑病变_黄斑前膜
右黄斑病变_黄斑裂孔
双屈光不正
[5, 6]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑牵拉综合征
['双老年性白内障', '双视网膜动脉硬化', '双玻璃体后脱离', '右底出血', '左视网膜分支静脉阻塞', '左黄斑病变_黄斑前膜', '左黄斑病变_黄斑牵拉综合征']
双老年性白内障
双视网膜动脉硬化
双玻璃体后脱离
右底出血
左视网膜分支静脉阻塞
左黄斑病变_黄斑前膜
左黄斑病变_黄斑牵拉综合征
右PVD!双IOL!双黄斑病变_黄斑变性_AMD!左黄斑病变_黄斑变性_AMD_湿性!_瘢痕期
['右PVD', '双IOL', '双黄斑病变_黄斑变性_AMD', '左黄斑病变_黄斑变性_AMD_湿性', '_瘢痕期']
右PVD
双IOL
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
左黄斑病变_黄斑变性_AMD_湿性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AM

[0, 1]
min_index:右黄斑病变_黄斑前膜,max_index:右黄斑病变_黄斑牵拉综合征
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑牵拉综合征', '双高血压性视网膜病变']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑牵拉综合征
双高血压性视网膜病变
[0, 1]
min_index:右黄斑病变_黄斑裂孔,max_index:右黄斑病变_黄斑前膜
['右黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔', '左屈光间质混浊']
右黄斑病变_黄斑前膜
右黄斑病变_黄斑裂孔
左屈光间质混浊
[0, 1]
左黄斑病变_黄斑水肿 左黄斑病变_黄斑变性_AMD_湿性待排
['左黄斑病变_黄斑水肿', '左黄斑病变_黄斑变性_AMD_湿性待排', '左RVO待排']
左黄斑病变_黄斑水肿
左黄斑病变_黄斑变性_AMD_湿性待排
删除字符串的起始索引：0
删除字符串的结束索引：24
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性
左RVO待排
[1, 2]
min_index:右黄斑病变_黄斑裂孔,max_index:右黄斑病变_黄斑前膜
['双糖网', '右黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔']
双糖网
右黄斑病变_黄斑前膜
右黄斑病变_黄斑裂孔
[0, 1]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑裂孔_假性
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔_假性', '双视网膜动脉硬化', '双白内障']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑裂孔_假性
删除字符串的起始索引：0
删除字符串的结束索引：10
删除多余字符之后的标签项——x：黄斑病变,黄斑裂孔,假性
双视网膜动脉硬化
双白内障
[0, 1]
左黄斑病变_黄斑裂孔 左黄斑病变_黄斑前膜
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑裂孔
双黄斑病变_黄斑前膜!右黄斑病变_黄斑裂孔
['双黄斑病变_黄斑前膜', '右黄斑病变_黄斑裂孔']
双黄斑病变_黄斑前膜
d=============黄斑病变,黄斑前膜
右黄斑病变_黄斑裂孔
r**************黄斑病变,黄斑前膜;黄斑病变,黄斑裂孔
黄斑病变,黄斑前膜;黄斑病变,黄斑裂孔__黄斑病变,黄斑前膜
[0, 1, 2]
左黄斑病变_黄斑前膜 右黄斑病变_黄斑裂

[2, 3]
左黄斑病变_黄斑出血 左黄斑继发黄斑病变_黄斑变性_AMD_湿性
['左玻璃体后脱离', '双高度近视视网膜改变', '左黄斑继发黄斑病变_黄斑变性_AMD_湿性', '左黄斑病变_黄斑出血']
左玻璃体后脱离
双高度近视视网膜改变
左黄斑继发黄斑病变_黄斑变性_AMD_湿性
删除字符串的起始索引：0
删除字符串的结束索引：24
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,湿性
左黄斑病变_黄斑出血
[0]
双黄斑病变_黄斑变性!左黄斑病变_黄斑前膜!双白内障!右翼状胬肉初发期
['双黄斑病变_黄斑变性', '左黄斑病变_黄斑前膜', '双白内障', '右翼状胬肉初发期']
双黄斑病变_黄斑变性
d=============黄斑病变,黄斑变性
左黄斑病变_黄斑前膜
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性;黄斑病变,黄斑前膜
双白内障
右翼状胬肉初发期
黄斑病变,黄斑变性__黄斑病变,黄斑变性;黄斑病变,黄斑前膜
[0, 1]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑牵拉综合征综合征
['左黄斑病变_黄斑牵拉综合征综合征', '左黄斑病变_黄斑前膜']
左黄斑病变_黄斑牵拉综合征综合征
左黄斑病变_黄斑前膜
[0, 1]
左黄斑病变_黄斑裂孔 左黄斑病变_黄斑前膜
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑裂孔
[0, 1]
左黄斑病变_黄斑前膜 左可疑黄斑病变_黄斑裂孔
['左黄斑病变_黄斑前膜', '左可疑黄斑病变_黄斑裂孔']
左黄斑病变_黄斑前膜
左可疑黄斑病变_黄斑裂孔
双黄斑病变_黄斑变性_AMD!左黄斑病变_黄斑变性_AMD_湿性
['双黄斑病变_黄斑变性_AMD', '左黄斑病变_黄斑变性_AMD_湿性']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
左黄斑病变_黄斑变性_AMD_湿性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,A

双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑变性_AMD_瘢痕期!左Drusen
['双黄斑病变_黄斑变性_AMD', '右黄斑病变_黄斑变性_AMD_瘢痕期', '左Drusen']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右黄斑病变_黄斑变性_AMD_瘢痕期
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,瘢痕期
左Drusen
删除字符串的起始索引：0
删除字符串的结束索引：38
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,瘢痕期
黄斑病变,黄斑变性,AMD,瘢痕期__黄斑病变,黄斑变性,AMD
[0, 1]
min_index:右黄斑病变_黄斑前膜,max_index:右黄斑病变_黄斑裂孔
['右黄斑病变_黄斑裂孔', '右黄斑病变_黄斑前膜', '右高血压视网膜病变']
右黄斑病变_黄斑裂孔
右黄斑病变_黄斑前膜
右高血压视网膜病变
[1, 2]
左黄斑病变 左黄斑病变_黄斑出血
双黄斑病变_黄斑裂孔!右黄斑病变_黄斑前膜
['双黄斑病变_黄斑裂孔', '右黄斑病变_黄斑前膜']
双黄斑病变_黄斑裂孔
d=============黄斑病变,黄斑裂孔
右黄斑病变_黄斑前膜
r**************黄斑病变,黄斑裂孔;黄斑病变,黄斑前膜
黄斑病变,黄斑裂孔;黄斑病变,黄斑前膜__黄斑病变,黄斑裂孔
双白内障!双黄斑病变_黄斑变性_AMD!右黄斑病变_黄斑前膜
['双白内障', '双黄斑病变_黄斑变性_AMD', '右黄斑病变_黄斑前膜']
双白内障
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右黄斑病变_黄斑前膜
r**************黄斑病变,黄斑变性,AMD;黄斑病变,黄斑前膜
黄斑病变,黄斑变性,AMD;黄斑病变,黄斑前膜__黄斑病变,黄斑变性,AMD
[0, 1]
左黄斑病变

[0, 1]
左黄斑病变_黄斑前膜 左黄斑病变_黄斑裂孔_假性待查
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔_假性待查', '双玻璃体浑浊待查']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑裂孔_假性待查
删除字符串的起始索引：0
删除字符串的结束索引：10
删除多余字符之后的标签项——x：黄斑病变,黄斑裂孔,假性
双玻璃体浑浊待查
双黄斑病变_黄斑变性_AMD!右瘢痕期!左黄斑病变_黄斑变性_AMD_干性!双视网膜动脉硬化
['双黄斑病变_黄斑变性_AMD', '右瘢痕期', '左黄斑病变_黄斑变性_AMD_干性', '双视网膜动脉硬化']
双黄斑病变_黄斑变性_AMD
d=============黄斑病变,黄斑变性
d=============黄斑病变,黄斑变性,AMD
右瘢痕期
左黄斑病变_黄斑变性_AMD_干性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD
l~~~~~~~~~~~~~~~黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性;黄斑病变,黄斑变性,AMD;黄斑病变,黄斑变性,AMD,干性
双视网膜动脉硬化
删除字符串的起始索引：0
删除字符串的结束索引：38
删除多余字符之后的标签项——x：黄斑病变,黄斑变性,AMD,干性
黄斑病变,黄斑变性,AMD__黄斑病变,黄斑变性,AMD,干性
[0, 1]
左黄斑病变_黄斑裂孔 左黄斑病变_黄斑前膜
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑裂孔', '双高血压性视网膜病变', '双屈光不正', '双老视']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑裂孔
双高血压性视网膜病变
双屈光不正
双老视
[0, 1]
左黄斑病变_黄斑出血 左黄斑病变_黄斑前膜
['左黄斑病变_黄斑前膜', '左黄斑病变_黄斑出血']
左黄斑病变_黄斑前膜
左黄斑病变_黄斑出血
双白内障!双黄斑病变_黄斑变性_AMD_干性!右黄斑病变_黄斑前膜!双高血压性视网膜病变
['双白内障', '双黄斑病变_黄斑变性_AMD_干性', '右黄斑病变_黄斑前膜', '双高血压性视网膜病变']
双白内障
双黄斑病变_黄斑变性_AMD_干性
d=============黄斑病变,黄斑变性
d===

In [89]:
# 校验标签无重复子集项
def check_duplicated_fuc(df,column_name):
    check_count = {}
    for i in range(df.shape[0]):
        str_i = str(df.at[i,column_name])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        else:
            for m in range(len(column_tw_fu)):
                check_count[column_tw_fu[m]] = str_i.count(column_tw_fu[m])
    #             print("重复字符串：{}".format(check_count))
                if check_count[column_tw_fu[m]] > 1:
                    print(str_i)

In [90]:
check_duplicated_fuc(df_amd,'left_黄斑病变')
check_duplicated_fuc(df_amd,'right_黄斑病变')

### '阅_gai'字段——含“黄斑病变”标签的数据容量

In [ ]:
print("'阅_gai'字段——含黄斑病变——数据容量:{}".format(df_amd[df_amd['阅_gai'].str.contains('黄斑病变')].shape[0] ))  # 含青光眼的数据容量

In [ ]:
# 去除多余的；
for i in range(10):
    df_amd['left_黄斑病变'].replace(to_replace=';;',value='',inplace=True)
    df_amd['right_黄斑病变'].replace(to_replace=';;',value='',inplace=True)
    # 使用正则表达式替换——去掉开头的；
    df_amd['left_黄斑病变'].replace(to_replace=r'^;',value='',regex=True,inplace=True)
    df_amd['right_黄斑病变'].replace(to_replace=r'^;',value='',regex=True,inplace=True)
    # 使用正则表达式替换——去掉结尾的；
    df_amd['left_黄斑病变'].replace(to_replace=r';$',value='',regex=True,inplace=True)
    df_amd['right_黄斑病变'].replace(to_replace=r';$',value='',regex=True,inplace=True)

In [ ]:
df_amd['left_黄斑病变'].value_counts()

In [ ]:
df_amd_jiao = df_amd[['阅片诊断','阅_gai','left_黄斑病变','right_黄斑病变']].drop_duplicates(keep='first')  # 去重
## 患者维度
print("'阅_gai'字段去重后——数据容量:{}".format(df_amd_jiao.shape[0]) )  # '阅_gai'字段去重后的数据容量
df_amd_jiao[:5]

In [ ]:
## 患者维度——含白内障
### '阅_gai'字段——含“白内障”标签的数据容量——去重后，唯一标签
df_amd_jiao_notduplicate = df_amd_jiao[df_amd_jiao['阅_gai'].str.contains('黄斑病变')]   # '阅_gai'字段去重后——含白内障的数据容量
print("'阅_gai'字段去重后_含黄斑病变——数据容量:{}".format(df_amd_jiao_notduplicate.shape[0]))

In [ ]:
df_last_01 = df_amd_jiao.sample(n=5000,replace=False,random_state=None,axis=0)

In [ ]:
df_last_01[df_last_01['阅_gai'].str.contains('黄斑病变')].shape[0]

In [ ]:
df_last_01[df_last_01['阅_gai'].str.contains('黄斑(.*)色素')].shape[0]

In [ ]:
output_path = r'/Users/edz/Documents/yandi_data/导出数据'
output_file = r'/黄斑病变_21570条_20200305.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_last_01.to_excel(writer,sheet_name='黄斑病变_5000',index=None)
df_amd_jiao_notduplicate.to_excel(writer,sheet_name='含黄斑病变_去重_11755',index=None)
df_amd_jiao.to_excel(writer,sheet_name='阅_gai_去重_60504',index=None)
writer.save()

In [ ]:
# 输出全量250625的糖网标签
df_amd_quan = df_amd[['来自检查id','阅_gai','left_黄斑病变','right_黄斑病变']]

In [ ]:
df_amd_quan.shape[0]

In [ ]:
# 导出新增疾病标签
output_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_ai_疾病标签'
output_file = r'/黄斑病变_quan_250625.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_amd_quan.to_excel(writer,sheet_name='黄斑病变'+str(df_amd_quan.shape[0]),index=None)
writer.save()